In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
import gc
import lightgbm as lgb
import module.train_log as log
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from keras.callbacks import EarlyStopping
from numba import jit
from tqdm import tqdm_notebook as tqdm
import seaborn as sns 
from datetime import datetime as dt
import pandas_profiling as pp
%matplotlib inline

Using TensorFlow backend.
C:\Users\southman\Anaconda3\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\southman\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\southman\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\southman\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\southman\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\southman\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
 

In [2]:
# 불러올 데이터의 타입과 컬럼을 정의한다.
def get_column_dtypes():
    return {
        'MachineIdentifier':                                    'object',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'}

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# 데이터를 불러온다.
def load_data():
    column_dtypes = get_column_dtypes()
    #use_columns = get_use_columns()
    tr_train = pd.read_csv('../input/train.csv', dtype=column_dtypes)
    X_test = pd.read_csv('../input/test.csv', dtype=column_dtypes)
    
    # 학습 데이터의 xy를 분리한다.
    y_train = tr_train.HasDetections
    X_train = tr_train.drop(['HasDetections'], axis=1)
    return X_train, y_train, X_test

In [414]:
def remove_columns(columns):
    X_train.drop(columns=columns, inplace=True)
    X_test.drop(columns=columns, inplace=True)

def get_missing_columns(tr_train, p=0.9):
    missing_columns = []
    for column in X_train.columns:
        if X_train[column].isna().mean() > p:
            missing_columns.append(column)
    return missing_columns

def remove_missing_columns(X_train, X_test, p=0.9):
    missing_columns = get_missing_columns(X_train, p)
    print("Missing Columns:", missing_columns)
    remove_columns(missing_columns)

In [489]:
def value_counts(column):
    train = tr_train[column].value_counts()
    train_sum = tr_train.groupby(column)['HasDetections'].sum()
    test = X_test[column].value_counts()
    df = pd.DataFrame({"train":train, "train_sum":train_sum, "test":test})
    df["train_ratio"] = df.train_sum / df.train
    return df

def value_counts_simple(column):
    train = X_train[column].value_counts()
    test = X_test[column].value_counts()
    df = pd.DataFrame({"train":train, "test":test})
    return df
    
def all_value_counts():
    for column in tqdm(X_train.columns):
        if column == 'HasDetections' or column == 'MachineIdentifier':
            continue
        print(column)
        display(value_counts(column))
        
def get_null_data(column):
    counts = value_counts_simple(column)
    return counts.loc[counts.train.isna() | counts.test.isna()]

def non_key_to_nan(column):
    nul_row = get_null_data(column)
    X_train.loc[tr_train[column].isin(nul_row.index), column] = np.nan
    X_test.loc[X_test[column].isin(nul_row.index), column] = np.nan
    if X_train[column].dtype.name == 'category':
        X_train[column] = X_train[column].cat.remove_unused_categories()
        X_test[column] = X_test[column].cat.remove_unused_categories()

def to_categorical(column):
    X_train[column] = X_train[column].astype("category")
    X_test[column] = X_test[column].astype("category")
    

def change_value(column, value, change_value):
    if X_train[column].dtype.name == 'category':
        if X_train[column].cat.categories.isin([change_value]).any() == False:
            X_train[column].cat.add_categories([change_value], inplace=True)
        if X_test[column].cat.categories.isin([change_value]).any() == False:
            X_test[column].cat.add_categories([change_value], inplace=True)
    X_train.loc[X_train[column] == value, column] = change_value
    X_test.loc[X_test[column] == value, column] = change_value
    if X_train[column].dtype.name == 'category':
        X_train[column] = X_train[column].cat.remove_unused_categories()
        X_test[column] = X_test[column].cat.remove_unused_categories()
        
# Target Encoder를 실행한다.
def to_target_encode(category_columns=None):
    global X_train, X_test
    if category_columns is None:
        category_columns = list(X_train.select_dtypes(include=['category']).columns)
    print("Columns to encode:", category_columns)
    enc = ce.TargetEncoder(cols=category_columns)
    enc.fit(X_train, y_train)
    print("Fitted")
    X_train = enc.transform(X_train)
    print("Transformed train data")
    X_test = enc.transform(X_test)
    print("Transformed test data")
    
    X_train.to_pickle("../result/X_train_target_encoded.pkl")
    X_test.to_pickle("../result/X_test_target_encoded.pkl")
    print("Saved to pickle data")
    return X_train, X_test

# Target Encoder를 실행한다.
def to_target_encode_each(category_columns=None):
    global X_train, X_test
    if category_columns is None:
        category_columns = list(X_train.select_dtypes(include=['category']).columns)
    for column in tqdm(category_columns):
        enc = ce.TargetEncoder(cols=[column])
        enc.fit(X_train, y_train)
        X_train = enc.transform(X_train)
        X_test = enc.transform(X_test)
        print("Encoded column:", column)
    return X_train, X_test

def to_label_encode(category_columns=None):
    global X_train, X_test
    if category_columns is None:
        category_columns = list(X_train.select_dtypes(include=['category']).columns)
    for column in tqdm(category_columns):
        fill_na([column])
        enc = preprocessing.LabelEncoder()
        enc.fit(np.unique(X_train[column].unique().tolist()+ X_test[column].unique().tolist()))
        X_train[column] = enc.transform(X_train[column].astype(str)) + 1
        X_test[column] = enc.transform(X_test[column].astype(str)) + 1
        print("Encoded column:", column)
    return X_train, X_test

In [ ]:
def add_feature_engine_version(X):
    X['EngineVersion_1'] = X.EngineVersion.str.split(".", expand=True)[2].astype('int16')
    X['EngineVersion_2'] = X.EngineVersion.str.split(".", expand=True)[3].astype('int8')

def add_feature_app_version(X):
    X['AppVersion_1'] = X.AppVersion.str.split(".", expand=True)[1].astype('int8')
    X['AppVersion_2'] = X.AppVersion.str.split(".", expand=True)[2].astype('int16')
    X['AppVersion_3'] = X.AppVersion.str.split(".", expand=True)[3].astype('int16')

def add_feature_av_sig_version(X):
    X['AvSigVersion_1'] = X.AvSigVersion.str.split(".", expand=True)[0].astype('int8')
    X['AvSigVersion_2'] = X.AvSigVersion.str.split(".", expand=True)[1]
    X.loc[X.AvSigVersion_2 == '2&#x17;3', 'AvSigVersion_2'] = '23'
    X['AvSigVersion_2'] = X.AvSigVersion_2.astype('int16')
    X['AvSigVersion_3'] = X.AvSigVersion.str.split(".", expand=True)[2].astype('int16')
    
def add_feature_os_ver(X):
    X['OsVer_1'] = X.OsVer.str.split(".", expand=True)[0].astype('int8')
    X['OsVer_2'] = X.OsVer.str.split(".", expand=True)[1].astype('int8')
    X['OsVer_3'] = X.OsVer.str.split(".", expand=True)[2].astype('int16')
    X['OsVer_4'] = X.OsVer.str.split(".", expand=True)[3].astype('int16')
    
def add_feature_os_build_lab(X):
    X['OsBuildLab_1'] = X.OsBuildLab.str.split(".", expand=True)[0]
    X.loc[X.OsBuildLab_1.isna(), 'OsBuildLab_1'] = '0'
    X['OsBuildLab_1'] = X.OsBuildLab_1.astype('int16')
    X['OsBuildLab_2'] = X.OsBuildLab.str.split(".", expand=True)[1]
    X.loc[X.OsBuildLab_2.isna(), 'OsBuildLab_2'] = '0'
    X.loc[X.OsBuildLab_2 == '1*amd64fre', 'OsBuildLab_2'] = '64'
    X['OsBuildLab_2'] = X.OsBuildLab_2.astype('int16')
    X['OsBuildLab_3'] = X.OsBuildLab.str.split(".", expand=True)[2].astype('category')
    X['OsBuildLab_4'] = X.OsBuildLab.str.split(".", expand=True)[3].astype('category')
    X['OsBuildLab_5'] = X.OsBuildLab.str.split(".", expand=True)[4].astype('category')
    
def add_feature_c_display_Inches(X):
    feature = 'Census_InternalPrimaryDiagonalDisplaySizeInInches_round'
    X[feature] = round(X['Census_InternalPrimaryDiagonalDisplaySizeInInches'])
    X.loc[X[feature].isna() ,feature] = 0
    X[feature] = X[feature].astype('int16')

def add_feature_c_os_version(X):
    X['Census_OSVersion_1'] = X.Census_OSVersion.str.split(".", expand=True)[0].astype('int8')
    X['Census_OSVersion_2'] = X.Census_OSVersion.str.split(".", expand=True)[1].astype('int8')
    X['Census_OSVersion_3'] = X.Census_OSVersion.str.split(".", expand=True)[2].astype('int16')
    X['Census_OSVersion_4'] = X.Census_OSVersion.str.split(".", expand=True)[3].astype('int16')
    
def add_feature_c_os_branch(X):
    X['Census_OSBranch_1'] = X.Census_OSBranch.str.split("_", expand=True)[0].astype('category')
    X['Census_OSBranch_2'] = X.Census_OSBranch.str.split("_", expand=True)[1].astype('category')

In [410]:
# https://www.kaggle.com/artgor/is-this-malware-eda-fe-and-lgb-updated
def add_feature_primary_drive_c_ratio(X):
    X['primary_drive_c_ratio'] = X['Census_SystemVolumeTotalCapacity']/ X['Census_PrimaryDiskTotalCapacity']
    
def add_feature_non_primary_drive(X):
    X['non_primary_drive_MB'] = X['Census_PrimaryDiskTotalCapacity'] - X['Census_SystemVolumeTotalCapacity']

def add_feature_aspect_ratio(X):
    X['aspect_ratio'] = X['Census_InternalPrimaryDisplayResolutionHorizontal']/ X['Census_InternalPrimaryDisplayResolutionVertical']

def add_feature_monitor_dims(X):
    X['monitor_dims'] = X['Census_InternalPrimaryDisplayResolutionHorizontal'].astype(str) + '*' + X['Census_InternalPrimaryDisplayResolutionVertical'].astype('str')
    X['monitor_dims'] = X['monitor_dims'].astype('category')
    
def add_feature_dpi(X):
    X['Census_InternalPrimaryDisplayResolutionHorizontal'] = X['Census_InternalPrimaryDisplayResolutionHorizontal'].astype("float32")
    X['Census_InternalPrimaryDisplayResolutionVertical'] = X['Census_InternalPrimaryDisplayResolutionVertical'].astype("float32")
    X['Census_InternalPrimaryDiagonalDisplaySizeInInches'] = X['Census_InternalPrimaryDiagonalDisplaySizeInInches'].astype("float32")
    
    X['dpi'] = ((X['Census_InternalPrimaryDisplayResolutionHorizontal']**2 + X['Census_InternalPrimaryDisplayResolutionVertical']**2)**.5)/(X['Census_InternalPrimaryDiagonalDisplaySizeInInches'])
    #X['dpi_square'] = X['dpi'] ** 2
    
def add_feature_mega_pixels(X):
    X['Census_InternalPrimaryDisplayResolutionHorizontal'] = X['Census_InternalPrimaryDisplayResolutionHorizontal'].astype("float32")
    X['Census_InternalPrimaryDisplayResolutionVertical'] = X['Census_InternalPrimaryDisplayResolutionVertical'].astype("float32")
    
    X['MegaPixels'] = (X['Census_InternalPrimaryDisplayResolutionHorizontal'] * X['Census_InternalPrimaryDisplayResolutionVertical'])/1e6


def add_feature_screen_area(X):
    X['Screen_Area'] = (X['aspect_ratio']* (X['Census_InternalPrimaryDiagonalDisplaySizeInInches']**2))/(X['aspect_ratio']**2 + 1)
    
def add_feature_ram_per_processor(X):
    X['ram_per_processor'] = X['Census_TotalPhysicalRAM']/ X['Census_ProcessorCoreCount']
    
def add_feature_new_num(X):
    X['new_num_0'] = X['Census_InternalPrimaryDiagonalDisplaySizeInInches'] / X['Census_ProcessorCoreCount']
    X['new_num_1'] = X['Census_ProcessorCoreCount'] * X['Census_InternalPrimaryDiagonalDisplaySizeInInches']

In [439]:
def add_features():
    add_feature_methods = [add_feature_engine_version, add_feature_app_version, add_feature_av_sig_version, add_feature_os_ver,
                          add_feature_os_build_lab, add_feature_c_display_Inches, add_feature_c_os_version, add_feature_c_os_branch,
                          add_feature_primary_drive_c_ratio, add_feature_aspect_ratio,
                          add_feature_monitor_dims, add_feature_dpi, add_feature_mega_pixels, add_feature_screen_area,
                          add_feature_ram_per_processor, add_feature_new_num]

    for method in tqdm(add_feature_methods):
        method(X_train)
        method(X_test)
    
    non_key_to_nan("OsBuildLab_3")
    non_key_to_nan("OsBuildLab_4")
    non_key_to_nan("OsBuildLab_5")
    non_key_to_nan("Census_OSBranch_1")
    non_key_to_nan("Census_OSBranch_2")
    non_key_to_nan("monitor_dims")
    
    reduce_mem_usage(X_train)
    reduce_mem_usage(X_test) 


In [470]:
def to_log(feature):
    X_train[feature] = np.log(X_train[feature])
    X_test[feature] = np.log(X_test[feature])
    
def fill_na(columns=None):
    if columns is None:
        columns = list(X_train.columns[X_train.isna().any()])
    for column in list(X_train.columns[X_train.isna().any()]):
        if  pd.api.types.is_categorical_dtype(X_train[column]):
            X_train[column] = X_train[column].cat.add_categories(['NA'])
            X_train[column].fillna('NA', inplace=True)
            X_test[column] = X_test[column].cat.add_categories(['NA'])
            X_test[column].fillna('NA', inplace=True)
        else:
            mean = X_train[column].mean()
            X_train[column].fillna(mean, inplace=True)
            X_test[column].fillna(mean, inplace=True)

    return X_train, X_test    

In [366]:
@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

def eval_auc(preds, dtrain):
    labels = dtrain.get_label()
    return 'auc', fast_auc(labels, preds), True

In [361]:
# ID 저장 및 제거 
def drop_id_n_get_test_id():
    ID_test = X_test.MachineIdentifier
    X_train.drop(['MachineIdentifier'], axis=1, inplace=True)
    X_test.drop(['MachineIdentifier'], axis=1, inplace=True)
    return ID_test

def get_base_params(random_state):
    return {   
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'nthread': 4,
        'learning_rate': 0.05,
        'max_depth': 5,
        'num_leaves': 40,
        'sub_feature': 0.9,
        'sub_row':0.9,
        'bagging_freq': 1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'random_state': random_state
        }

In [363]:
def train_model(x, y, lgb_params, 
                number_of_folds=5, 
                evaluation_metric='auc', 
                save_feature_importances=False, 
                early_stopping_rounds=50, 
                num_round = 50,
                random_state = 42,
                shuffle = True,
                break_first=False):
    cross_validator = StratifiedKFold(n_splits=number_of_folds,
                                  random_state=random_state,
                                  shuffle=shuffle)
    
    validation_scores = []
    classifier_models = []
    feature_importance_df = pd.DataFrame()
    for fold_index, (train_index, validation_index) in enumerate(cross_validator.split(x, y)):
        gc.collect()
        x_train, x_validation = x.iloc[train_index], x.iloc[validation_index]
        y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
    
        x_train_columns = x_train.columns
        trn_data = lgb.Dataset(x_train, label=y_train)
        val_data = lgb.Dataset(x_validation, label=y_validation)
        classifier_model = lgb.train(lgb_params,
                                     trn_data,
                                     num_round,
                                     valid_sets=[trn_data, val_data],
                                     verbose_eval=100,
                                     early_stopping_rounds=early_stopping_rounds,
                                     feval=eval_auc
                                     )

        classifier_models.append(classifier_model)
        
        predictions = classifier_model.predict(x_validation, num_iteration=classifier_model.best_iteration)
        false_positive_rate, recall, thresholds = metrics.roc_curve(y_validation, predictions)
        score = metrics.auc(false_positive_rate, recall)
        validation_scores.append(score)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = x_train_columns
        fold_importance_df["importance"] = classifier_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = fold_index + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        if break_first:
            break
        
    if save_feature_importances:
        cols = (feature_importance_df[["feature", "importance"]]
                .groupby("feature")
                .mean()
                .sort_values(by="importance", ascending=False)[:1000].index)

        best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

        plt.figure(figsize=(14, 25))
        sns.barplot(x="importance",
                    y="feature",
                    data=best_features.sort_values(by="importance",
                                                   ascending=False))
        plt.title('LightGBM Features (avg over folds)')
        plt.tight_layout()
        plt.savefig('lgbm_importances.png')

        # mean_gain = feature_importances[['gain', 'feature']].groupby('feature').mean()
        # feature_importances['mean_gain'] = feature_importances['feature'].map(mean_gain['gain'])
        #
        # temp = feature_importances.sort_values('mean_gain', ascending=False)
        best_features.sort_values(by="importance", ascending=False) \
            .groupby("feature") \
            .mean() \
            .sort_values(by="importance", ascending=False) \
            .to_csv('feature_importances_new.csv', index=True)

    score = sum(validation_scores) / len(validation_scores)
    return classifier_models, score

def get_predicts(models, X_test):
    models_cnt = len(models)
    predictions = np.zeros(len(X_test))
    for model in tqdm(models):
        current_pred = model.predict(X_test, num_iteration=model.best_iteration)
        predictions += current_pred / models_cnt
    return predictions

In [163]:
tr_train, X_test = load_data()

Mem. usage decreased to 2009.00 Mb (0.0% reduction)
Mem. usage decreased to 1761.13 Mb (0.0% reduction)


In [164]:
tr_train.dtypes

MachineIdentifier                            object
ProductName                                category
EngineVersion                              category
AppVersion                                 category
AvSigVersion                               category
IsBeta                                         int8
RtpStateBitfield                            float16
IsSxsPassiveMode                               int8
DefaultBrowsersIdentifier                   float16
AVProductStatesIdentifier                   float32
AVProductsInstalled                         float16
AVProductsEnabled                           float16
HasTpm                                         int8
CountryIdentifier                             int16
CityIdentifier                              float32
OrganizationIdentifier                      float16
GeoNameIdentifier                           float16
LocaleEnglishNameIdentifier                    int8
Platform                                   category
Processor   

In [165]:
tr_train.HasDetections.value_counts()

0    4462591
1    4458892
Name: HasDetections, dtype: int64

In [166]:
#remove_missing_columns(tr_train, X_test)

### ProductName

In [167]:
tr_train.ProductName.describe()

count          8921483
unique               6
top       win8defender
freq           8826520
Name: ProductName, dtype: object

In [168]:
value_counts('ProductName')

train  train_sum     test  train_ratio
fep                  7        3.0        7     0.428571
mse              94873    45961.0    55946     0.484448
mseprerelease       53       26.0       34     0.490566
scep                22       10.0       16     0.454545
win8defender   8826520  4412891.0  7797245     0.499958
windowsintune        8        1.0        5     0.125000

### EngineVersion

In [169]:
tr_train.EngineVersion.describe()

count         8921483
unique             70
top       1.1.15200.1
freq          3845067
Name: EngineVersion, dtype: object

In [170]:
value_counts('EngineVersion')

train  train_sum       test  train_ratio
1.1.10302.0        3.0        1.0        2.0     0.333333
1.1.10401.0        5.0        3.0        2.0     0.600000
1.1.10701.0        1.0        1.0        NaN     1.000000
1.1.11104.0        1.0        1.0        1.0     1.000000
1.1.11202.0        1.0        1.0        NaN     1.000000
1.1.11302.0        NaN        NaN        1.0          NaN
1.1.11400.0        6.0        2.0        4.0     0.333333
1.1.11502.0        8.0        3.0        1.0     0.375000
1.1.11602.0        9.0        6.0        2.0     0.666667
1.1.11701.0       86.0       53.0       61.0     0.616279
1.1.11804.0        4.0        1.0        4.0     0.250000
1.1.11903.0       13.0        8.0        6.0     0.615385
1.1.12002.0        7.0        2.0        4.0     0.285714
1.1.12101.0       70.0       47.0       43.0     0.671429
1.1.12205.0        6.0        2.0        3.0     0.333333
1.1.12300.0        3.0        2.0        2.0     0.666667
1.1.12400.0       36.0       15.0       22.0     0.416667
1.1.12505.0       14.0        4.0       12.0     0.285714
1.1.12603.0       20.0       10.0       14.0     0.500000
1.1.12706.0       18.0        5.0       13.0     0.277778
1.1.12802.0        1.0        0.0        NaN     0.000000
1.1.12804.0       18.0        8.0        9.0     0.444444
1.1.12805.0     2427.0      991.0     1733.0     0.408323
1.1.12902.0     5395.0     2036.0     3108.0     0.377386
1.1.13000.0     4095.0     1478.0     2335.0     0.360928
1.1.13102.0        4.0        1.0        2.0     0.250000
1.1.13103.0     4440.0     1654.0     2477.0     0.372523
1.1.13202.0     4696.0     1713.0     2764.0     0.364779
1.1.13301.0        2.0        0.0        NaN     0.000000
1.1.13303.0     9315.0     3179.0     5303.0     0.341278
...                ...        ...        ...          ...
1.1.14003.0    14291.0     5611.0     7146.0     0.392625
1.1.14102.0       63.0       14.0       28.0     0.222222
1.1.14103.0       86.0       19.0       50.0     0.220930
1.1.14104.0    93926.0    41295.0    72795.0     0.439655
1.1.14201.0      220.0       65.0       94.0     0.295455
1.1.14202.0    15449.0     6349.0     9365.0     0.410965
1.1.14303.0      267.0       87.0      151.0     0.325843
1.1.14305.0     5095.0     2136.0     3115.0     0.419235
1.1.14306.0    24079.0     9331.0    13256.0     0.387516
1.1.14405.2    36341.0    13972.0    19278.0     0.384469
1.1.14500.2      279.0       85.0      146.0     0.304659
1.1.14500.5    45855.0    17899.0    23934.0     0.390339
1.1.14600.4   160585.0    70201.0   138514.0     0.437158
1.1.14700.3     1328.0      492.0      616.0     0.370482
1.1.14700.4     1105.0      440.0      534.0     0.398190
1.1.14700.5    46777.0    15165.0    21307.0     0.324198
1.1.14800.1      971.0      288.0      477.0     0.296601
1.1.14800.3   136476.0    42273.0    55992.0     0.309747
1.1.14901.3     1631.0      506.0      802.0     0.310239
1.1.14901.4   212408.0    64245.0    75808.0     0.302460
1.1.15000.1     2472.0      729.0      920.0     0.294903
1.1.15000.2   265218.0    91183.0    66904.0     0.343804
1.1.15100.1  3675915.0  2031651.0   158036.0     0.552693
1.1.15200.1  3845067.0  1890743.0   366085.0     0.491732
1.1.15300.5    68716.0    36099.0     4480.0     0.525336
1.1.15300.6   120295.0    59240.0  3101305.0     0.492456
1.1.15400.3        NaN        NaN    22260.0          NaN
1.1.15400.4        NaN        NaN  2106236.0          NaN
1.1.15400.5        NaN        NaN  1491273.0          NaN
1.1.9700.0        10.0        6.0       11.0     0.600000

[74 rows x 4 columns]

In [171]:
get_null_data('EngineVersion')

train  train_sum       test  train_ratio
1.1.10701.0    1.0        1.0        NaN          1.0
1.1.11202.0    1.0        1.0        NaN          1.0
1.1.11302.0    NaN        NaN        1.0          NaN
1.1.12802.0    1.0        0.0        NaN          0.0
1.1.13301.0    2.0        0.0        NaN          0.0
1.1.15400.3    NaN        NaN    22260.0          NaN
1.1.15400.4    NaN        NaN  2106236.0          NaN
1.1.15400.5    NaN        NaN  1491273.0          NaN

In [172]:
non_key_to_nan('EngineVersion')

### AppVersion

In [173]:
tr_train.AppVersion.describe()

count             8921483
unique                110
top       4.18.1807.18075
freq              5139224
Name: AppVersion, dtype: object

In [174]:
value_counts('AppVersion')

train  train_sum      test  train_ratio
4.10.14393.0       77244.0    40926.0   66911.0     0.529828
4.10.14393.1066     7172.0     3416.0    4758.0     0.476297
4.10.14393.1198    23568.0    11413.0   14771.0     0.484258
4.10.14393.1532      116.0       70.0     216.0     0.603448
4.10.14393.1593     9602.0     4716.0    5309.0     0.491148
4.10.14393.1613    13525.0     6508.0    8177.0     0.481183
4.10.14393.1794    44109.0    21269.0   28618.0     0.482192
4.10.14393.2248      290.0      129.0     193.0     0.444828
4.10.14393.2273      880.0      582.0     971.0     0.661364
4.10.14393.2457       85.0       52.0     257.0     0.611765
4.10.14393.2608        NaN        NaN      21.0          NaN
4.10.14393.726       239.0      145.0     232.0     0.606695
4.10.14393.953      7321.0     3535.0    5286.0     0.482858
4.10.205.0          1113.0      454.0     651.0     0.407907
4.10.207.0             NaN        NaN       2.0          NaN
4.10.209.0        272455.0   136731.0  155962.0     0.501848
4.11.15063.0       69250.0    39138.0   71211.0     0.565170
4.11.15063.1154       18.0        7.0       4.0     0.388889
4.11.15063.1155    44238.0    18964.0   30882.0     0.428681
4.11.15063.447     91694.0    42533.0   56750.0     0.463858
4.11.15063.994        71.0       24.0      40.0     0.338028
4.12.16299.0           2.0        2.0       4.0     1.000000
4.12.16299.15     359871.0   164577.0  267102.0     0.457322
4.12.17007.17121      13.0        4.0       9.0     0.307692
4.12.17007.17123   13828.0     6514.0    7863.0     0.471073
4.12.17007.18011   61806.0    29217.0   33486.0     0.472721
4.12.17007.18021       9.0        5.0       1.0     0.555556
4.12.17007.18022  118027.0    49202.0   59238.0     0.416871
4.13.17133.1           NaN        NaN       1.0          NaN
4.13.17134.1      257270.0   134242.0  231117.0     0.521794
...                    ...        ...       ...          ...
4.8.10240.17533       25.0       13.0      21.0     0.520000
4.8.10240.17609      375.0      211.0     235.0     0.562667
4.8.10240.17770      144.0       78.0      88.0     0.541667
4.8.10240.17797      656.0      296.0     327.0     0.451220
4.8.10240.17861      638.0      337.0     260.0     0.528213
4.8.10240.17889     1622.0      978.0     290.0     0.602959
4.8.10240.17914     1771.0     1207.0      83.0     0.681536
4.8.10240.17918      497.0      308.0      40.0     0.619718
4.8.10240.17943        1.0        0.0       NaN     0.000000
4.8.10240.17946     4786.0     2889.0    5454.0     0.603636
4.8.10240.18036        NaN        NaN    1284.0          NaN
4.8.203.0              1.0        0.0       1.0     0.000000
4.8.204.0           1471.0      670.0    1174.0     0.455472
4.8.207.0           4153.0     1824.0    2488.0     0.439201
4.9.10586.0       111797.0    57206.0  115686.0     0.511695
4.9.10586.1045     10559.0     4689.0    7069.0     0.444076
4.9.10586.1106    203525.0    90466.0  176442.0     0.444496
4.9.10586.1177        21.0       10.0       8.0     0.476190
4.9.10586.456         46.0       30.0      54.0     0.652174
4.9.10586.461          NaN        NaN       1.0          NaN
4.9.10586.494      24725.0    12848.0   23271.0     0.519636
4.9.10586.589      13268.0     5769.0    9947.0     0.434806
4.9.10586.672      22428.0    10029.0   17133.0     0.447164
4.9.10586.839       4668.0     1969.0    3274.0     0.421808
4.9.10586.873       4439.0     1876.0    3111.0     0.422618
4.9.10586.916       5677.0     2453.0    4017.0     0.432094
4.9.10586.962       4789.0     1999.0    3359.0     0.417415
4.9.10586.965       5235.0     2254.0    3634.0     0.430564
4.9.218.0           5859.0     2652.0    3988.0     0.452637
4.9.219.0              3.0        0.0       4.0     0.000000

[124 rows x 4 columns]

In [175]:
non_key_to_nan('AppVersion')

### AvSigVersion

In [176]:
tr_train.AvSigVersion.describe()

count          8921483
unique            8531
top       1.273.1420.0
freq            102317
Name: AvSigVersion, dtype: object

In [177]:
value_counts('AvSigVersion')

train  train_sum     test  train_ratio
0.0.0.0        68.0        8.0     67.0     0.117647
1.155.266.0    10.0        6.0     11.0     0.600000
1.167.387.0     3.0        1.0      2.0     0.333333
1.169.1625.0    1.0        1.0      NaN     1.000000
1.169.2478.0    NaN        NaN      1.0          NaN
1.169.55.0      4.0        2.0      1.0     0.500000
1.177.229.0     1.0        1.0      NaN     1.000000
1.187.1121.0    1.0        1.0      NaN     1.000000
1.187.322.0     NaN        NaN      1.0          NaN
1.189.189.0     1.0        1.0      NaN     1.000000
1.191.897.0     NaN        NaN      1.0          NaN
1.193.185.0     1.0        0.0      NaN     0.000000
1.193.1928.0    1.0        0.0      NaN     0.000000
1.193.3377.0    NaN        NaN      1.0          NaN
1.193.3388.0    NaN        NaN      1.0          NaN
1.193.418.0     4.0        2.0      2.0     0.500000
1.195.109.0     3.0        1.0      1.0     0.333333
1.195.1107.0    1.0        1.0      NaN     1.000000
1.195.2226.0    1.0        0.0      NaN     0.000000
1.195.3092.0    2.0        1.0      NaN     0.500000
1.195.3225.0    1.0        0.0      NaN     0.000000
1.197.1076.0    1.0        1.0      NaN     1.000000
1.197.1237.0    1.0        0.0      NaN     0.000000
1.197.2142.0    1.0        1.0      NaN     1.000000
1.197.2441.0    1.0        1.0      NaN     1.000000
1.197.2843.0    1.0        0.0      NaN     0.000000
1.197.52.0      4.0        3.0      2.0     0.750000
1.199.1324.0    1.0        1.0      NaN     1.000000
1.199.1515.0    NaN        NaN      1.0          NaN
1.199.1615.0   80.0       50.0     59.0     0.625000
...             ...        ...      ...          ...
1.281.739.0     NaN        NaN  42143.0          NaN
1.281.748.0     NaN        NaN  13892.0          NaN
1.281.750.0     NaN        NaN  12104.0          NaN
1.281.756.0     NaN        NaN  10783.0          NaN
1.281.758.0     NaN        NaN  10652.0          NaN
1.281.76.0      NaN        NaN   2684.0          NaN
1.281.760.0     NaN        NaN  11843.0          NaN
1.281.765.0     NaN        NaN  10632.0          NaN
1.281.767.0     NaN        NaN  11497.0          NaN
1.281.774.0     NaN        NaN  13067.0          NaN
1.281.777.0     NaN        NaN  14626.0          NaN
1.281.78.0      NaN        NaN   2183.0          NaN
1.281.782.0     NaN        NaN  16049.0          NaN
1.281.785.0     NaN        NaN  16927.0          NaN
1.281.788.0     NaN        NaN  16411.0          NaN
1.281.790.0     NaN        NaN  18525.0          NaN
1.281.796.0     NaN        NaN  17053.0          NaN
1.281.801.0     NaN        NaN  15401.0          NaN
1.281.803.0     NaN        NaN  15048.0          NaN
1.281.806.0     NaN        NaN  12848.0          NaN
1.281.812.0     NaN        NaN   9077.0          NaN
1.281.815.0     NaN        NaN   7597.0          NaN
1.281.818.0     NaN        NaN    100.0          NaN
1.281.82.0      NaN        NaN   2191.0          NaN
1.281.821.0     NaN        NaN   4629.0          NaN
1.281.85.0      NaN        NaN   2308.0          NaN
1.281.87.0      NaN        NaN   3039.0          NaN
1.281.90.0      NaN        NaN   5971.0          NaN
1.281.95.0      NaN        NaN   3477.0          NaN
1.281.98.0      NaN        NaN   3608.0          NaN

[9623 rows x 4 columns]

In [178]:
non_key_to_nan('AvSigVersion')

### IsBeta

In [179]:
tr_train.IsBeta.describe()

count    8.921483e+06
mean     7.509962e-06
std      2.740421e-03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: IsBeta, dtype: float64

In [415]:
value_counts('IsBeta')

NameError: name 'tr_train' is not defined

In [416]:
remove_columns("IsBeta")

### RtpStateBitfield

In [181]:
tr_train.RtpStateBitfield.describe()

count    8889165.0
mean           NaN
std            0.0
min            0.0
25%            7.0
50%            7.0
75%            7.0
max           35.0
Name: RtpStateBitfield, dtype: float64

In [187]:
value_counts('RtpStateBitfield')

train  train_sum     test  train_ratio
0.0    190701    71641.0   162531     0.375672
1.0      1625      649.0     1425     0.399385
3.0      3029      907.0     1921     0.299439
5.0     20328     3069.0    12845     0.150974
7.0   8651487  4351523.0  7619055     0.502980
8.0     21974    16426.0    23229     0.747520
35.0       21       19.0       24     0.904762

In [185]:
to_categorical('RtpStateBitfield')

In [186]:
non_key_to_nan('RtpStateBitfield')

### IsSxsPassiveMode

In [188]:
value_counts('IsSxsPassiveMode')

train  train_sum     test  train_ratio
IsSxsPassiveMode                                          
0                 8766840  4402017.0  7728637     0.502121
1                  154643    56875.0   124616     0.367783

### DefaultBrowsersIdentifier

In [189]:
value_counts("DefaultBrowsersIdentifier")

train  train_sum     test  train_ratio
1.0        39.0       20.0     22.0     0.512821
2.0         3.0        2.0      1.0     0.666667
3.0         2.0        2.0      5.0     1.000000
4.0         NaN        NaN      1.0          NaN
5.0         1.0        0.0      NaN     0.000000
6.0         1.0        1.0      NaN     1.000000
7.0         6.0        5.0      NaN     0.833333
8.0        13.0        7.0      6.0     0.538462
9.0        19.0       10.0     10.0     0.526316
10.0      255.0      102.0     65.0     0.400000
11.0        1.0        1.0      NaN     1.000000
12.0        NaN        NaN      1.0          NaN
14.0        1.0        1.0      NaN     1.000000
15.0        1.0        1.0      NaN     1.000000
16.0        3.0        2.0      NaN     0.666667
17.0        NaN        NaN      1.0          NaN
18.0        NaN        NaN      3.0          NaN
19.0        1.0        0.0      NaN     0.000000
20.0        1.0        0.0      NaN     0.000000
23.0       10.0        9.0     10.0     0.900000
24.0      152.0       46.0     46.0     0.302632
25.0        1.0        0.0      NaN     0.000000
26.0        5.0        3.0      NaN     0.600000
27.0        1.0        0.0      NaN     0.000000
28.0        5.0        2.0      1.0     0.400000
29.0        1.0        0.0      NaN     0.000000
30.0        NaN        NaN    436.0          NaN
31.0        NaN        NaN      1.0          NaN
32.0        3.0        1.0      3.0     0.333333
33.0        1.0        1.0      NaN     1.000000
...         ...        ...      ...          ...
3150.0      NaN        NaN      1.0          NaN
3152.0      3.0        1.0      1.0     0.333333
3154.0      1.0        0.0      NaN     0.000000
3156.0      NaN        NaN      3.0          NaN
3158.0      1.0        0.0      NaN     0.000000
3160.0      3.0        2.0      2.0     0.666667
3162.0      NaN        NaN      1.0          NaN
3164.0    108.0       66.0     67.0     0.611111
3166.0      2.0        2.0      NaN     1.000000
3168.0      2.0        2.0      3.0     1.000000
3170.0      NaN        NaN      1.0          NaN
3172.0      3.0        2.0      NaN     0.666667
3174.0      NaN        NaN      1.0          NaN
3176.0  24222.0     9590.0  13135.0     0.395921
3178.0      NaN        NaN      1.0          NaN
3180.0      3.0        2.0      4.0     0.666667
3184.0     12.0        6.0      7.0     0.500000
3186.0      7.0        5.0      4.0     0.714286
3188.0      1.0        1.0      2.0     1.000000
3190.0      NaN        NaN     10.0          NaN
3192.0      5.0        1.0      1.0     0.200000
3194.0     18.0       13.0     28.0     0.722222
3196.0  42694.0    23438.0  37181.0     0.548976
3198.0      NaN        NaN     31.0          NaN
3200.0      NaN        NaN      1.0          NaN
3204.0     10.0        6.0      3.0     0.600000
3206.0      1.0        1.0      1.0     1.000000
3208.0      3.0        1.0     11.0     0.333333
3212.0      4.0        2.0      NaN     0.500000
3214.0      NaN        NaN      2.0          NaN

[2177 rows x 4 columns]

In [190]:
non_key_to_nan('DefaultBrowsersIdentifier')

In [191]:
to_categorical('DefaultBrowsersIdentifier')

### AVProductStatesIdentifier                   

In [192]:
value_counts("AVProductStatesIdentifier")              

train  train_sum  test  train_ratio
2.0        NaN        NaN   1.0          NaN
3.0        1.0        1.0   1.0     1.000000
4.0        NaN        NaN   2.0          NaN
5.0        NaN        NaN   1.0          NaN
6.0        1.0        0.0   NaN     0.000000
7.0        1.0        1.0   NaN     1.000000
9.0        NaN        NaN   1.0          NaN
12.0       NaN        NaN   1.0          NaN
13.0       NaN        NaN   1.0          NaN
14.0       1.0        0.0   NaN     0.000000
15.0       NaN        NaN   1.0          NaN
16.0      13.0        5.0   4.0     0.384615
17.0       1.0        1.0   NaN     1.000000
20.0       1.0        0.0   NaN     0.000000
23.0       2.0        1.0   1.0     0.500000
24.0       1.0        1.0   NaN     1.000000
25.0       1.0        0.0   NaN     0.000000
26.0       NaN        NaN   1.0          NaN
27.0       NaN        NaN   1.0          NaN
28.0       NaN        NaN   2.0          NaN
31.0       1.0        0.0   1.0     0.000000
33.0       NaN        NaN   1.0          NaN
34.0       2.0        0.0   NaN     0.000000
35.0       1.0        0.0   NaN     0.000000
36.0       NaN        NaN   1.0          NaN
38.0       1.0        1.0   NaN     1.000000
39.0      96.0       40.0  69.0     0.416667
40.0       1.0        0.0   NaN     0.000000
41.0       1.0        1.0   5.0     1.000000
43.0       5.0        1.0   2.0     0.200000
...        ...        ...   ...          ...
70444.0    4.0        0.0   1.0     0.000000
70445.0    NaN        NaN   1.0          NaN
70446.0    1.0        1.0   NaN     1.000000
70447.0    1.0        0.0   NaN     0.000000
70449.0    NaN        NaN   1.0          NaN
70450.0    2.0        1.0   NaN     0.500000
70452.0    NaN        NaN   1.0          NaN
70454.0    NaN        NaN   1.0          NaN
70458.0    2.0        0.0   NaN     0.000000
70459.0    2.0        0.0   NaN     0.000000
70460.0    1.0        0.0   1.0     0.000000
70461.0    1.0        0.0   2.0     0.000000
70462.0    NaN        NaN   1.0          NaN
70468.0    2.0        1.0   NaN     0.500000
70469.0  133.0       16.0  90.0     0.120301
70471.0    NaN        NaN   1.0          NaN
70474.0    NaN        NaN   1.0          NaN
70475.0    1.0        0.0   NaN     0.000000
70476.0    2.0        2.0   NaN     1.000000
70479.0    NaN        NaN   1.0          NaN
70481.0    NaN        NaN   1.0          NaN
70482.0    NaN        NaN   1.0          NaN
70486.0  101.0       38.0  58.0     0.376238
70490.0    3.0        0.0   5.0     0.000000
70492.0   12.0        6.0   1.0     0.500000
70493.0    1.0        0.0   1.0     0.000000
70496.0    1.0        0.0   NaN     0.000000
70498.0    1.0        1.0   NaN     1.000000
70502.0    NaN        NaN   1.0          NaN
70507.0    1.0        0.0   NaN     0.000000

[39831 rows x 4 columns]

In [193]:
non_key_to_nan('AVProductStatesIdentifier')

In [194]:
to_categorical('AVProductStatesIdentifier')

### AVProductsInstalled

In [195]:
value_counts("AVProductsInstalled")  

train  train_sum       test  train_ratio
0.0        1        0.0        NaN     0.000000
1.0  6208893  3406078.0  5849609.0     0.548581
2.0  2459008   975996.0  1825605.0     0.396906
3.0   208103    60682.0   147704.0     0.291596
4.0     8757     2371.0     6218.0     0.270755
5.0      471      125.0      329.0     0.265393
6.0       28        6.0       21.0     0.214286
7.0        1        1.0        NaN     1.000000

### AVProductsEnabled

In [196]:
value_counts("AVProductsEnabled")  

train  train_sum     test  train_ratio
0.0    25958     9184.0    17603     0.353802
1.0  8654101  4367169.0  7657407     0.504636
2.0   198652    66831.0   148869     0.336422
3.0     6075     1924.0     5193     0.316708
4.0      453      149.0      396     0.328918
5.0       23        2.0       18     0.086957

### HasTpm

In [197]:
value_counts("HasTpm")  

train  train_sum     test  train_ratio
0   107316    50966.0    65052     0.474915
1  8814167  4407926.0  7788201     0.500096

### CountryIdentifier

In [198]:
value_counts("CountryIdentifier")  

train  train_sum    test  train_ratio
1      2141      945.0    1573     0.441383
2     66243    30708.0   54965     0.463566
3      4722     2345.0    4507     0.496612
4      2210      874.0    1776     0.395475
5       459      222.0     381     0.483660
6     10024     4812.0    9257     0.480048
7     14467     8459.0   13899     0.584710
8     12677     6311.0   10502     0.497831
9    172594    87455.0  126392     0.506709
10      327      135.0     287     0.412844
11    54946    24307.0   36651     0.442380
12    81538    37457.0   64472     0.459381
13     2693     1078.0    1704     0.400297
14      446      129.0     301     0.289238
15    10058     4997.0   10609     0.496818
16    16062     7567.0   13622     0.471112
17     6076     2426.0    4310     0.399276
18    57691    30755.0   64899     0.533099
19    66748    30186.0   47313     0.452238
20     3065     1615.0    3117     0.526917
21    52367    29173.0   49100     0.557087
22    12862     5940.0   10468     0.461826
23      934      506.0     952     0.541756
24     3840     2045.0    3496     0.532552
25      944      254.0     913     0.269068
26     7299     3606.0    5338     0.494040
27    24142    14218.0   23856     0.588932
28      553      220.0     369     0.397830
29   347991   175846.0  299643     0.505318
30     6690     2624.0    4326     0.392227
..      ...        ...     ...          ...
193     207      120.0     353     0.579710
194    2416     1243.0    2531     0.514487
195  131685    72021.0  122142     0.546919
196     681      355.0     849     0.521292
197     793      443.0     701     0.558638
198     656      387.0     876     0.589939
199   20456     9537.0   21121     0.466220
200     355      179.0     264     0.504225
201  220622   112664.0  216462     0.510665
202   17330     7539.0   13451     0.435026
203  158058    78463.0  122879     0.496419
204   10211     4649.0    8501     0.455293
205  117245    61197.0  131148     0.521958
206    5424     2496.0    5230     0.460177
207  211645   102423.0  191072     0.483938
208   25493    11753.0   18648     0.461029
209    3603     1852.0    4065     0.514016
210    1657      696.0    1424     0.420036
211   42544    21621.0   37820     0.508203
212     299      118.0     291     0.394649
213    1303      348.0     879     0.267076
214  191269   116083.0  191298     0.606910
215     543      296.0     375     0.545120
216     379      147.0     253     0.387863
217     120       76.0     318     0.633333
218    3086     1799.0    3777     0.582955
219    2395      881.0    1420     0.367850
220   81459    37754.0   61997     0.463472
221    9041     4207.0    7479     0.465325
222   13947     7147.0   13142     0.512440

[222 rows x 4 columns]

In [199]:
to_categorical('CountryIdentifier')

### CityIdentifier

In [200]:
value_counts("CityIdentifier")  

train  train_sum   test  train_ratio
1.0         NaN        NaN   11.0          NaN
2.0         NaN        NaN    1.0          NaN
3.0         NaN        NaN    1.0          NaN
4.0         NaN        NaN    9.0          NaN
5.0         2.0        1.0   10.0     0.500000
7.0        33.0       20.0   23.0     0.606061
8.0         3.0        0.0    1.0     0.000000
9.0         7.0        5.0    NaN     0.714286
10.0        NaN        NaN    1.0          NaN
11.0        6.0        3.0    6.0     0.500000
12.0       22.0       12.0   35.0     0.545455
14.0        NaN        NaN   42.0          NaN
15.0        2.0        0.0    2.0     0.000000
16.0        2.0        1.0    4.0     0.500000
19.0        4.0        3.0    1.0     0.750000
20.0        NaN        NaN    1.0          NaN
21.0        NaN        NaN    5.0          NaN
22.0        3.0        2.0    7.0     0.666667
23.0       20.0        7.0   15.0     0.350000
24.0       16.0       13.0   12.0     0.812500
25.0        1.0        0.0    1.0     0.000000
26.0        NaN        NaN    2.0          NaN
27.0        NaN        NaN    2.0          NaN
28.0        NaN        NaN    3.0          NaN
29.0        7.0        4.0    5.0     0.571429
30.0       42.0       23.0   22.0     0.547619
31.0        2.0        1.0    1.0     0.500000
32.0       11.0        7.0    4.0     0.636364
34.0       38.0       20.0   30.0     0.526316
35.0        1.0        1.0    NaN     1.000000
...         ...        ...    ...          ...
167931.0    3.0        0.0    8.0     0.000000
167932.0    5.0        4.0    8.0     0.800000
167933.0   50.0       25.0   50.0     0.500000
167934.0  168.0       78.0  190.0     0.464286
167935.0    1.0        1.0    1.0     1.000000
167936.0    NaN        NaN    1.0          NaN
167937.0    NaN        NaN    1.0          NaN
167938.0   32.0       15.0   23.0     0.468750
167939.0   27.0       11.0   19.0     0.407407
167940.0    1.0        1.0    NaN     1.000000
167941.0   48.0       20.0   31.0     0.416667
167942.0    4.0        2.0   13.0     0.500000
167943.0    1.0        0.0    NaN     0.000000
167944.0   16.0        8.0   17.0     0.500000
167945.0    4.0        4.0    4.0     1.000000
167946.0   60.0       31.0   81.0     0.516667
167948.0    NaN        NaN    2.0          NaN
167950.0    3.0        0.0    NaN     0.000000
167951.0    NaN        NaN    1.0          NaN
167952.0    2.0        0.0    2.0     0.000000
167953.0   79.0       48.0   85.0     0.607595
167954.0   20.0        9.0   40.0     0.450000
167955.0    4.0        2.0    NaN     0.500000
167956.0    NaN        NaN    1.0          NaN
167957.0    2.0        1.0    4.0     0.500000
167958.0    1.0        1.0    1.0     1.000000
167959.0    8.0        1.0    7.0     0.125000
167960.0    NaN        NaN    1.0          NaN
167961.0    1.0        0.0    2.0     0.000000
167962.0   16.0        9.0   14.0     0.562500

[130523 rows x 4 columns]

In [201]:
non_key_to_nan('CityIdentifier')

In [202]:
to_categorical('CityIdentifier')

### OrganizationIdentifier

In [203]:
value_counts("OrganizationIdentifier")

train  train_sum       test  train_ratio
1.0       893.0      425.0      939.0     0.475924
2.0      2595.0      997.0     2233.0     0.384200
3.0       331.0      133.0      269.0     0.401813
4.0      1385.0      609.0     1389.0     0.439711
5.0      1990.0      906.0     1916.0     0.455276
6.0       412.0      133.0      379.0     0.322816
7.0        98.0       37.0      103.0     0.377551
8.0       723.0      417.0      809.0     0.576763
9.0         NaN        NaN        1.0          NaN
10.0     1083.0      463.0      967.0     0.427516
11.0    19436.0    10043.0    16871.0     0.516722
12.0        7.0        2.0       16.0     0.285714
13.0        NaN        NaN        7.0          NaN
14.0     4713.0     1931.0     3547.0     0.409718
15.0       13.0        6.0        6.0     0.461538
16.0      242.0      102.0      257.0     0.421488
17.0        1.0        1.0        NaN     1.000000
18.0  1764175.0   892153.0  1552377.0     0.505705
19.0      172.0       59.0      230.0     0.343023
20.0      915.0      547.0     1171.0     0.597814
21.0      397.0      106.0      187.0     0.267003
22.0      418.0      208.0      437.0     0.497608
23.0       20.0        6.0       10.0     0.300000
25.0       12.0        3.0        3.0     0.250000
26.0      160.0       49.0       84.0     0.306250
27.0  4196457.0  2088092.0  3605244.0     0.497585
28.0     1591.0      804.0     1756.0     0.505343
29.0      135.0       65.0      129.0     0.481481
30.0       64.0       23.0       78.0     0.359375
31.0      398.0      179.0      348.0     0.449749
32.0     4045.0     2083.0     3790.0     0.514957
33.0     2896.0     1183.0     2654.0     0.408494
34.0        2.0        1.0        3.0     0.500000
35.0       32.0       11.0       26.0     0.343750
36.0     3909.0     2133.0     3553.0     0.545664
37.0    19398.0     9033.0    16743.0     0.465667
38.0        1.0        1.0     1267.0     1.000000
39.0      413.0      191.0      410.0     0.462470
40.0     1648.0      719.0     1405.0     0.436286
41.0       77.0       41.0       91.0     0.532468
42.0      132.0       72.0      135.0     0.545455
43.0       60.0       28.0       52.0     0.466667
44.0      150.0       80.0      155.0     0.533333
45.0       73.0       32.0       70.0     0.438356
46.0    10974.0     5643.0     9658.0     0.514215
47.0      385.0      188.0      420.0     0.488312
48.0    63845.0    33211.0    77154.0     0.520182
49.0    13627.0     6221.0    12060.0     0.456520
50.0    45502.0    27752.0    45856.0     0.609907
51.0      917.0      416.0     1364.0     0.453653
52.0     3043.0     1501.0     2495.0     0.493263

In [204]:
non_key_to_nan('OrganizationIdentifier')

In [205]:
to_categorical('OrganizationIdentifier')

### GeoNameIdentifier

In [206]:
value_counts("GeoNameIdentifier")

train  train_sum       test  train_ratio
1.0       1070.0      590.0      979.0     0.551402
2.0       2236.0     1171.0     2285.0     0.523703
3.0       4205.0     2082.0     4903.0     0.495125
4.0        161.0       75.0      170.0     0.465839
5.0        605.0      251.0      433.0     0.414876
6.0       3209.0     1617.0     2749.0     0.503895
7.0        321.0      174.0      273.0     0.542056
8.0         60.0       26.0       53.0     0.433333
9.0        711.0      305.0      532.0     0.428973
10.0    143023.0    73206.0   104650.0     0.511848
11.0      4492.0     2827.0     4553.0     0.629341
12.0      1154.0      474.0      767.0     0.410745
13.0         1.0        0.0        2.0     0.000000
14.0         1.0        0.0        1.0     0.000000
15.0     78629.0    34543.0    62833.0     0.439316
16.0     40883.0    17852.0    27249.0     0.436661
17.0      2642.0     1438.0     2830.0     0.544285
18.0      2642.0     1053.0     1587.0     0.398562
19.0      6431.0     3139.0     5235.0     0.488104
20.0         NaN        NaN        1.0          NaN
21.0     10666.0     6016.0    12206.0     0.564035
22.0      2737.0     1103.0     1901.0     0.402996
23.0     11796.0     6647.0    14584.0     0.563496
24.0     57541.0    25429.0    40445.0     0.441928
25.0      1970.0      953.0     1437.0     0.483756
26.0       894.0      464.0      851.0     0.519016
27.0       434.0      101.0      397.0     0.232719
28.0       333.0      196.0      318.0     0.588589
29.0     29253.0    15102.0    26946.0     0.516255
30.0     15395.0     9307.0    14920.0     0.604547
...          ...        ...        ...          ...
267.0   215812.0   111478.0   211515.0     0.516551
268.0      255.0      141.0      296.0     0.552941
269.0      410.0      169.0      334.0     0.412195
270.0      189.0      103.0      174.0     0.544974
271.0      428.0      192.0      438.0     0.448598
272.0       45.0       18.0       23.0     0.400000
273.0     1983.0      950.0     1934.0     0.479072
274.0    59877.0    32779.0    70642.0     0.547439
275.0    44909.0    21053.0    37230.0     0.468792
276.0   296774.0   138394.0   240133.0     0.466328
277.0  1531929.0   784129.0  1407739.0     0.511857
278.0        1.0        0.0        NaN     0.000000
279.0        1.0        0.0        NaN     0.000000
280.0        6.0        5.0        8.0     0.833333
281.0      487.0      255.0      536.0     0.523614
282.0    19002.0     8974.0    14085.0     0.472266
283.0     1039.0      560.0     1166.0     0.538980
284.0      399.0      211.0      334.0     0.528822
285.0      208.0       98.0      216.0     0.471154
286.0     1844.0      878.0      660.0     0.476139
287.0    63751.0    36547.0    62711.0     0.573277
288.0      832.0      349.0      732.0     0.419471
289.0      274.0      109.0      233.0     0.397810
290.0        4.0        3.0        4.0     0.750000
291.0      148.0       68.0      137.0     0.459459
292.0       13.0        9.0       15.0     0.692308
293.0     1374.0      799.0     1639.0     0.581514
294.0     3727.0     1797.0     3058.0     0.482157
295.0     8141.0     4176.0     7658.0     0.512959
296.0      102.0       50.0       91.0     0.490196

[295 rows x 4 columns]

In [207]:
non_key_to_nan('GeoNameIdentifier')

In [208]:
to_categorical('GeoNameIdentifier')

### LocaleEnglishNameIdentifier

In [209]:
value_counts("LocaleEnglishNameIdentifier")

train  train_sum      test  train_ratio
-128      23.0       11.0      25.0     0.478261
-127    1337.0      661.0    1222.0     0.494390
-126      83.0       41.0      81.0     0.493976
-125      12.0        6.0       8.0     0.500000
-124     219.0       97.0     141.0     0.442922
-123      10.0        5.0       8.0     0.500000
-122  172519.0    82849.0  161325.0     0.480231
-121      49.0       24.0      59.0     0.489796
-120      89.0       47.0      69.0     0.528090
-119       4.0        3.0       1.0     0.750000
-118      14.0        7.0       2.0     0.500000
-117   10042.0     4464.0    7541.0     0.444533
-116   15387.0     7698.0   11615.0     0.500292
-115      13.0        3.0      13.0     0.230769
-114     261.0       77.0     137.0     0.295019
-113    1812.0      955.0    1417.0     0.527042
-112     117.0       51.0      47.0     0.435897
-111      59.0       31.0      57.0     0.525424
-110       3.0        1.0       1.0     0.333333
-109    1586.0      748.0    1063.0     0.471627
-108      58.0       29.0      58.0     0.500000
-107     630.0      252.0     436.0     0.400000
-106      13.0        9.0      17.0     0.692308
-105      15.0        8.0       8.0     0.533333
-104      14.0        5.0      16.0     0.357143
-103       2.0        1.0       1.0     0.500000
-102     418.0      244.0     514.0     0.583732
-101      21.0       15.0       7.0     0.714286
-100       3.0        1.0       7.0     0.333333
-99       36.0       20.0      27.0     0.555556
...        ...        ...       ...          ...
 98       29.0        9.0      27.0     0.310345
 99        3.0        2.0       1.0     0.666667
 100     267.0      151.0     285.0     0.565543
 101    1352.0      654.0    1026.0     0.483728
 102   40201.0    17485.0   26526.0     0.434939
 103  180913.0    81496.0  142648.0     0.450471
 104     248.0       63.0     164.0     0.254032
 105    1346.0      349.0     851.0     0.259287
 106   29211.0    11228.0   19177.0     0.384376
 107   57983.0    27369.0   46935.0     0.472018
 108      92.0       26.0      72.0     0.282609
 109      22.0       10.0      19.0     0.454545
 110      31.0       16.0      23.0     0.516129
 111      47.0       19.0      43.0     0.404255
 112      46.0       25.0      40.0     0.543478
 113   50801.0    26120.0   39947.0     0.514163
 114    3633.0     1863.0    2466.0     0.512799
 115   87899.0    47462.0   74441.0     0.539961
 116    3457.0     1281.0    2095.0     0.370553
 117      50.0       22.0      39.0     0.440000
 118   52718.0    30798.0   45136.0     0.584203
 119      10.0        4.0       8.0     0.400000
 120       6.0        4.0       2.0     0.666667
 121     270.0      118.0     196.0     0.437037
 122       5.0        4.0      10.0     0.800000
 123      18.0        9.0      11.0     0.500000
 124  195088.0    94665.0  167661.0     0.485243
 125    1910.0     1031.0    1163.0     0.539791
 126   84194.0    36474.0   64088.0     0.433214
 127      20.0        9.0      12.0     0.450000

[254 rows x 4 columns]

In [210]:
to_categorical('LocaleEnglishNameIdentifier')

### Platform

In [211]:
value_counts("Platform")

train  train_sum     test  train_ratio
windows10    8618715  4309629.0  7675480     0.500032
windows2016    14371     5024.0    10986     0.349593
windows7       93889    45678.0    55240     0.486511
windows8      194508    98561.0   111547     0.506720

### Processor

In [212]:
value_counts("Processor")

train  train_sum     test  train_ratio
arm64      346        5.0      211     0.014451
x64    8105435  4145493.0  7173360     0.511446
x86     815702   313394.0   679682     0.384202

### OsVer

In [388]:
value_counts("OsVer")

train  train_sum     test  train_ratio
10.0.0.0    8632545  4314355.0  7686083     0.499778
10.0.0.1          5        2.0        5     0.400000
10.0.0.112       10        4.0        2     0.400000
10.0.0.3          2        0.0        1     0.000000
10.0.1.0        141       79.0      112     0.560284
10.0.1.144        1        1.0        1     1.000000
10.0.1.44         3        2.0        1     0.666667
10.0.16.0         9        3.0        5     0.333333
10.0.16.36        6        3.0        2     0.500000
10.0.2.0         30       17.0       15     0.566667
10.0.2.86         2        1.0        2     0.500000
10.0.3.0        225      130.0      167     0.577778
10.0.3.80         2        0.0        2     0.000000
10.0.32.0         2        2.0        1     1.000000
10.0.32.72       23       11.0       16     0.478261
10.0.4.0         11        8.0        9     0.727273
10.0.5.0         18       11.0       16     0.611111
10.0.5.18        12        6.0        2     0.500000
10.0.6.0          3        0.0        3     0.000000
10.0.7.0         15        8.0        4     0.533333
10.0.8.0          5        3.0        3     0.600000
10.0.80.0         4        1.0        5     0.250000
6.1.0.0         582      289.0      572     0.496564
6.1.0.112         1        0.0        1     0.000000
6.1.1.0       93268    45358.0    54653     0.486319
6.1.2.0           2        1.0        1     0.500000
6.1.3.0          30       25.0       11     0.833333
6.3.0.0      194447    98525.0   111520     0.506693
6.3.0.112         1        0.0        2     0.000000
6.3.1.0          22       14.0        7     0.636364
6.3.3.0          24       14.0       14     0.583333
6.3.32.72         2        1.0        1     0.500000
6.3.4.0           3        2.0        1     0.666667

In [214]:
non_key_to_nan('OsVer')

In [419]:
remove_columns('OsVer')

### OsBuild

In [215]:
value_counts("OsBuild")

train  train_sum       test  train_ratio
7600       583.0      291.0      573.0     0.499142
7601     93306.0    45387.0    54667.0     0.486432
9600    194508.0    98561.0   111547.0     0.506720
10240   270192.0   131471.0   248346.0     0.486584
10586   411606.0   191739.0   367350.0     0.465831
14393   730819.0   337835.0   575569.0     0.462269
15063   780270.0   373652.0   616089.0     0.478875
16299  2503681.0  1232131.0  1690582.0     0.492128
17133        6.0        3.0        2.0     0.500000
17134  3915521.0  2038918.0  3893188.0     0.520727
17604        3.0        1.0        4.0     0.333333
17618        8.0        5.0        3.0     0.625000
17623        1.0        1.0        2.0     1.000000
17627       31.0       14.0       21.0     0.451613
17634       47.0       27.0       49.0     0.574468
17639       49.0       18.0       47.0     0.367347
17643       20.0        6.0       12.0     0.300000
17650       64.0       33.0       53.0     0.515625
17653        NaN        NaN        2.0          NaN
17655       19.0        5.0       11.0     0.263158
17661      128.0       42.0       58.0     0.328125
17666      209.0       81.0      102.0     0.387560
17672      370.0      115.0      170.0     0.310811
17677      315.0      122.0       98.0     0.387302
17682      266.0       92.0       89.0     0.345865
17686      570.0      226.0      227.0     0.396491
17689        1.0        0.0        NaN     0.000000
17692     3184.0     1412.0      341.0     0.443467
17694        1.0        0.0        NaN     0.000000
17704       49.0       16.0       23.0     0.326531
...          ...        ...        ...          ...
18237      167.0       70.0       82.0     0.419162
18238        1.0        0.0        1.0     0.000000
18240        5.0        0.0        NaN     0.000000
18241        1.0        0.0        NaN     0.000000
18242      103.0       52.0      178.0     0.504854
18244        1.0        0.0        NaN     0.000000
18247        NaN        NaN      273.0          NaN
18248        NaN        NaN        1.0          NaN
18249        NaN        NaN        3.0          NaN
18252        NaN        NaN     3246.0          NaN
18253        NaN        NaN        2.0          NaN
18255        NaN        NaN        4.0          NaN
18256        NaN        NaN        1.0          NaN
18259        NaN        NaN        8.0          NaN
18260        NaN        NaN        5.0          NaN
18262        NaN        NaN      982.0          NaN
18264        NaN        NaN        2.0          NaN
18265        NaN        NaN        2.0          NaN
18266        NaN        NaN        1.0          NaN
18267        NaN        NaN      461.0          NaN
18269        NaN        NaN        1.0          NaN
18270        NaN        NaN        2.0          NaN
18272        NaN        NaN      765.0          NaN
18277        NaN        NaN      459.0          NaN
18279        NaN        NaN        3.0          NaN
18280        NaN        NaN        1.0          NaN
18282        NaN        NaN     1124.0          NaN
18285        NaN        NaN        2.0          NaN
18287        NaN        NaN        2.0          NaN
18289        NaN        NaN        3.0          NaN

[101 rows x 4 columns]

In [216]:
non_key_to_nan('OsBuild')

In [217]:
to_categorical('OsBuild')

### OsSuite

In [218]:
value_counts("OsSuite")

train  train_sum       test  train_ratio
16       731      440.0      662.0     0.601915
18         1        1.0        3.0     1.000000
49        17        9.0       18.0     0.529412
144       34       18.0       20.0     0.529412
256  3346251  1718312.0  3303044.0     0.513504
272    12092     4056.0     9106.0     0.335428
274       39       11.0       23.0     0.282051
305      662      279.0      595.0     0.421450
307        2        0.0        3.0     0.000000
400      793      210.0      556.0     0.264817
402        1        1.0        1.0     1.000000
528        1        1.0        NaN     1.000000
768  5560661  2735465.0  4539020.0     0.491932
784      198       89.0      202.0     0.449495

In [219]:
non_key_to_nan('OsSuite')

In [220]:
to_categorical('OsSuite')

### OsPlatformSubRelease

In [221]:
value_counts("OsPlatformSubRelease")

train  train_sum     test  train_ratio
prers5        20992     8904.0   295341     0.424162
rs1          730819   337835.0   575569     0.462269
rs2          780270   373652.0   616089     0.478875
rs3         2503681  1232131.0  1690582     0.492128
rs4         3915526  2038921.0  3893189     0.520727
th1          270192   131471.0   248346     0.486584
th2          411606   191739.0   367350     0.465831
windows7      93889    45678.0    55240     0.486511
windows8.1   194508    98561.0   111547     0.506720

### OsBuildLab

In [222]:
value_counts("OsBuildLab")

train  train_sum  \
10240.16384.amd64fre.th1.150709-1700                24254.0    11666.0   
10240.16384.x86fre.th1.150709-1700                   2311.0      982.0   
10240.16387.amd64fre.th1_st1.150711-1429               96.0       33.0   
10240.16387.x86fre.th1_st1.150711-1429                  3.0        0.0   
10240.16390.x86fre.th1_st1.150714-1601                  1.0        0.0   
10240.16393.amd64fre.th1_st1.150717-1719             5900.0     2765.0   
10240.16393.x86fre.th1_st1.150717-1719               1141.0      525.0   
10240.16412.amd64fre.th1.150729-1800                  871.0      417.0   
10240.16412.x86fre.th1.150729-1800                     51.0       27.0   
10240.16425.amd64fre.th1.150802-1600                    2.0        1.0   
10240.16430.amd64fre.th1.150807-2049                  847.0      381.0   
10240.16430.x86fre.th1.150807-2049                     96.0       46.0   
10240.16431.amd64fre.th1.150810-2333                  778.0      353.0   
10240.16431.x86fre.th1.150810-2333                    233.0       94.0   
10240.16463.amd64fre.th1.150819-1946                 3931.0     1930.0   
10240.16463.x86fre.th1.150819-1946                    621.0      270.0   
10240.16515.amd64fre.th1.150916-2039                  952.0      456.0   
10240.16515.x86fre.th1.150916-2039                    147.0       40.0   
10240.16545.amd64fre.th1.150930-1750                  338.0      158.0   
10240.16545.x86fre.th1.150930-1750                     98.0       40.0   
10240.16590.amd64fre.th1_st1.151104-1714              382.0      185.0   
10240.16590.x86fre.th1_st1.151104-1714                 49.0       18.0   
10240.16644.amd64fre.th1.160104-1507                  425.0      200.0   
10240.16644.x86fre.th1.160104-1507                     55.0       17.0   
10240.16724.amd64fre.th1_st1.160222-1812              533.0      231.0   
10240.16724.x86fre.th1_st1.160222-1812                 91.0       29.0   
10240.16841.amd64fre.th1_st1.160408-1853              844.0      367.0   
10240.16841.x86fre.th1_st1.160408-1853                133.0       26.0   
10240.17022.amd64fre.th1_st1.160623-1726              507.0      212.0   
10240.17022.x86fre.th1_st1.160623-1726                 95.0       29.0   
...                                                     ...        ...   
9600.18821.amd64fre.winblue_ltsb.170914-0600         9537.0     3889.0   
9600.18821.x86fre.winblue_ltsb.170914-0600            531.0      173.0   
9600.18895.amd64fre.winblue_ltsb.180101-1800         1945.0      827.0   
9600.18895.x86fre.winblue_ltsb.180101-1800            185.0       70.0   
9600.18896.amd64fre.winblue_ltsb_escrow.180108-...     52.0       32.0   
9600.18896.x86fre.winblue_ltsb_escrow.180108-1534       1.0        1.0   
9600.18931.amd64fre.winblue_ltsb.180201-0600         2359.0      979.0   
9600.18931.x86fre.winblue_ltsb.180201-0600            204.0       77.0   
9600.18946.amd64fre.winblue_ltsb_escrow.180302-...   3097.0     1156.0   
9600.18946.x86fre.winblue_ltsb_escrow.180302-1800     235.0       61.0   
9600.18969.amd64fre.winblue_ltsb.180309-0600         4650.0     1726.0   
9600.18969.x86fre.winblue_ltsb.180309-0600            362.0      107.0   
9600.19000.amd64fre.winblue_ltsb.180410-0600         8155.0     3100.0   
9600.19000.x86fre.winblue_ltsb.180410-0600            647.0      188.0   
9600.19035.amd64fre.winblue_ltsb_escrow.180522-...  13994.0     5811.0   
9600.19035.x86fre.winblue_ltsb_escrow.180522-1700     957.0      256.0   
9600.19038.amd64fre.winblue_ltsb_escrow.180608-...    176.0       75.0   
9600.19038.x86fre.winblue_ltsb_escrow.180608-1416      17.0        7.0   
9600.19067.amd64fre.winblue_ltsb_escrow.180619-...  55853.0    32698.0   
9600.19067.x86fre.winblue_ltsb_escrow.180619-2033    3134.0     1307.0   
9600.19101.amd64fre.winblue_ltsb_escrow.180718-...  56036.0    31012.0   
9600.19101.x86fre.winblue_ltsb_escrow.180718-1800    3119.0     1190.0   
9600.19125.amd64fre.winblue_ltsb.180812-0703        16254.

In [223]:
non_key_to_nan('OsBuildLab')

### SkuEdition

In [224]:
value_counts("SkuEdition")

train  train_sum     test  train_ratio
Cloud               5589     2143.0     5639     0.383432
Education          40694    21445.0    36874     0.526982
Enterprise         34357    17753.0    31700     0.516721
Enterprise LTSB    20702    10820.0    20132     0.522655
Home             5514341  2714523.0  4513114     0.492266
Invalid            78054    36500.0    48013     0.467625
Pro              3224164  1654726.0  3195422     0.513226
Server              3582      982.0     2359     0.274149

### IsProtected

In [225]:
value_counts("IsProtected")

train  train_sum     test  train_ratio
0.0   483157   184253.0   432661     0.381352
1.0  8402282  4261098.0  7396943     0.507136

### AutoSampleOptIn

In [226]:
value_counts("AutoSampleOptIn")

train  train_sum     test  train_ratio
AutoSampleOptIn                                          
0                8921225  4458751.0  7853091     0.499791
1                    258      141.0      162     0.546512

In [422]:
remove_columns("AutoSampleOptIn")

### PuaMode

In [227]:
value_counts("PuaMode")

train  train_sum  test  train_ratio
PuaMode                                     
on        2307     1699.0  2178     0.736454
audit        2        2.0    10     1.000000

### SMode

In [228]:
value_counts("SMode")

train  train_sum     test  train_ratio
SMode                                          
0.0    8379843  4234781.0  2015562     0.505353
1.0       3881      650.0     6419     0.167483

### IeVerIdentifier

In [229]:
value_counts("IeVerIdentifier")

train  train_sum   test  train_ratio
1.0     18.0        8.0   17.0     0.444444
2.0      2.0        0.0    NaN     0.000000
6.0      NaN        NaN    2.0          NaN
7.0      NaN        NaN    1.0          NaN
8.0     12.0        4.0   13.0     0.333333
9.0      7.0        4.0    4.0     0.571429
10.0     7.0        5.0    5.0     0.714286
11.0     3.0        2.0    2.0     0.666667
12.0     5.0        1.0    1.0     0.200000
13.0     2.0        2.0    4.0     1.000000
14.0     3.0        2.0    4.0     0.666667
15.0    13.0        3.0   10.0     0.230769
16.0     7.0        5.0    8.0     0.714286
17.0     7.0        2.0   11.0     0.285714
18.0     6.0        2.0    8.0     0.333333
19.0     5.0        3.0    3.0     0.600000
20.0     2.0        0.0    2.0     0.000000
21.0     4.0        2.0    1.0     0.500000
22.0     1.0        1.0    NaN     1.000000
24.0     NaN        NaN    1.0          NaN
25.0     1.0        1.0    NaN     1.000000
27.0     2.0        1.0    3.0     0.500000
29.0     1.0        1.0    NaN     1.000000
32.0     1.0        1.0    1.0     1.000000
33.0     2.0        0.0    1.0     0.000000
34.0    11.0        7.0   14.0     0.636364
36.0     1.0        1.0    1.0     1.000000
38.0     1.0        0.0    2.0     0.000000
39.0   415.0      181.0  255.0     0.436145
40.0     1.0        0.0    NaN     0.000000
...      ...        ...    ...          ...
397.0    4.0        2.0    2.0     0.500000
398.0   76.0       30.0   65.0     0.394737
399.0    4.0        2.0    4.0     0.500000
400.0    2.0        1.0    3.0     0.500000
401.0    2.0        0.0    1.0     0.000000
402.0    1.0        0.0    NaN     0.000000
403.0    5.0        4.0    NaN     0.800000
404.0    1.0        0.0    6.0     0.000000
405.0    3.0        2.0    4.0     0.666667
406.0   11.0        3.0    2.0     0.272727
407.0    4.0        2.0    5.0     0.500000
408.0    7.0        4.0    5.0     0.571429
409.0    3.0        2.0    8.0     0.666667
410.0    2.0        1.0    NaN     0.500000
411.0    3.0        1.0    6.0     0.333333
412.0    1.0        0.0    1.0     0.000000
413.0    1.0        1.0    1.0     1.000000
414.0    1.0        0.0    NaN     0.000000
415.0    1.0        1.0    4.0     1.000000
416.0    1.0        0.0    1.0     0.000000
417.0    NaN        NaN    1.0          NaN
418.0    3.0        1.0    5.0     0.333333
419.0    NaN        NaN    1.0          NaN
420.0    NaN        NaN    1.0          NaN
421.0    1.0        0.0    2.0     0.000000
423.0   14.0       10.0   24.0     0.714286
425.0    1.0        0.0    2.0     0.000000
427.0    3.0        2.0    2.0     0.666667
428.0    4.0        1.0    1.0     0.250000
429.0  268.0       96.0  171.0     0.358209

[350 rows x 4 columns]

In [230]:
non_key_to_nan('IeVerIdentifier')

In [231]:
to_categorical('IeVerIdentifier')

### SmartScreen

In [255]:
value_counts("SmartScreen")

train  train_sum     test  train_ratio
&#x01;            335      199.0      267     0.594030
&#x02;            416      219.0      404     0.526442
0                   3        2.0        3     0.666667
Block           22533    11698.0    21242     0.519150
ExistsNotSet  1046183   844650.0   600446     0.807364
Off            187907    92777.0   163161     0.493739
On                878      493.0      939     0.561503
Prompt          34536    16446.0    28890     0.476199
RequireAdmin  4316194  1887931.0  3413571     0.437406
Warn           135484    78101.0   125926     0.576459

In [250]:
change_value("SmartScreen", "BLOCK", "Block")
change_value("SmartScreen", "OFF", "Off")
change_value("SmartScreen", "off", "Off")
change_value("SmartScreen", "ON", "On")
change_value("SmartScreen", "on", "On")
change_value("SmartScreen", "Promprt", "Prompt")
change_value("SmartScreen", "Promt", "Prompt")
change_value("SmartScreen", "prompt", "Prompt")
change_value("SmartScreen", "warn", "Warn")
change_value("SmartScreen", "RequiredAdmin", "RequireAdmin")
change_value("SmartScreen", "requireAdmin", "RequireAdmin")
change_value("SmartScreen", "requireadmin", "RequireAdmin")

In [254]:
non_key_to_nan('SmartScreen')

### Firewall

In [256]:
value_counts("Firewall")

train  train_sum     test  train_ratio
0.0   189119    92590.0   165426     0.489586
1.0  8641014  4321114.0  7629355     0.500070

### UacLuaenable

In [259]:
value_counts("UacLuaenable")

train  train_sum     test  train_ratio
0.0          53851    27028.0    59605     0.501903
1.0        8856517  4425785.0  7785557     0.499721
2.0             30       10.0       20     0.333333
3.0              6        4.0        9     0.666667
5.0              2        2.0        1     1.000000
48.0           206      110.0      167     0.533981
49.0            17        4.0       15     0.235294
6357062.0       13        6.0       12     0.461538

In [258]:
non_key_to_nan('UacLuaenable')

In [260]:
to_categorical('UacLuaenable')

### Census_MDC2FormFactor

In [263]:
value_counts("Census_MDC2FormFactor")

train  train_sum     test  train_ratio
AllInOne       292077   150581.0   255555     0.515552
Convertible    405378   201394.0   338763     0.496805
Desktop       1951086  1019851.0  1862386     0.522709
Detachable     298233   111996.0   245938     0.375532
IoTOther            1        0.0        1     0.000000
LargeServer       875      221.0      578     0.252571
LargeTablet     67121    20886.0    54204     0.311169
MediumServer     3385      998.0     2784     0.294830
Notebook      5723319  2876153.0  4929536     0.502532
PCOther        139955    67932.0   135453     0.485385
ServerOther        30        9.0       23     0.300000
SmallServer      8630     2996.0     6933     0.347161
SmallTablet     31393     5875.0    21098     0.187144

In [262]:
non_key_to_nan("Census_MDC2FormFactor")

### Census_DeviceFamily

In [264]:
value_counts("Census_DeviceFamily")

train  train_sum     test  train_ratio
Census_DeviceFamily                                          
Windows.Desktop      8907053  4453851.0  7842175     0.500036
Windows.Server         14410     5034.0    11053     0.349341
Windows                   20        7.0       25     0.350000

In [420]:
remove_columns("Census_DeviceFamily")

### Census_OEMNameIdentifier

In [269]:
value_counts("Census_OEMNameIdentifier")

train  train_sum  test  train_ratio
1.0         3        1.0     1     0.333333
9.0         1        0.0     1     0.000000
15.0        2        2.0     1     1.000000
20.0        2        2.0     1     1.000000
23.0        2        1.0     2     0.500000
47.0        3        1.0     3     0.333333
74.0      264      129.0   276     0.488636
77.0        1        0.0     1     0.000000
81.0        4        2.0     2     0.500000
82.0      121       37.0   109     0.305785
83.0        2        1.0     3     0.500000
84.0        6        1.0     6     0.166667
85.0       35       13.0    38     0.371429
86.0      227      107.0   179     0.471366
94.0        2        1.0     1     0.500000
96.0        4        1.0     3     0.250000
98.0        6        4.0     6     0.666667
104.0       1        1.0     1     1.000000
111.0       1        1.0     2     1.000000
112.0       1        0.0     1     0.000000
163.0       4        0.0     9     0.000000
164.0       6        0.0     2     0.000000
165.0      90       25.0    52     0.277778
167.0      20        9.0    20     0.450000
168.0       1        0.0     1     0.000000
169.0      15        3.0    10     0.200000
170.0       1        1.0     1     1.000000
171.0       6        2.0     2     0.333333
173.0       4        0.0     2     0.000000
174.0       3        1.0     3     0.333333
...       ...        ...   ...          ...
5952.0    746      315.0   644     0.422252
5960.0     16        3.0    20     0.187500
5968.0    106       28.0   172     0.264151
5972.0      3        1.0     2     0.333333
5976.0      2        1.0     1     0.500000
5980.0      2        0.0     1     0.000000
5984.0      1        1.0     1     1.000000
5992.0      1        1.0     1     1.000000
6000.0      5        2.0     3     0.400000
6016.0      2        1.0     2     0.500000
6024.0      1        1.0     2     1.000000
6032.0      2        1.0     1     0.500000
6036.0      2        1.0     1     0.500000
6040.0      3        0.0     1     0.000000
6044.0      1        0.0     1     0.000000
6048.0      1        0.0     2     0.000000
6056.0      3        0.0     2     0.000000
6060.0      1        0.0     1     0.000000
6064.0    575      264.0   592     0.459130
6072.0      1        0.0     1     0.000000
6076.0     28       14.0    30     0.500000
6080.0      2        0.0     2     0.000000
6088.0     14        7.0    16     0.500000
6092.0      1        1.0     1     1.000000
6096.0      2        1.0     1     0.500000
6120.0      2        2.0     1     1.000000
6132.0      3        2.0     1     0.666667
6136.0     26       14.0    22     0.538462
6140.0     33       17.0    40     0.515152
6144.0     44       22.0    32     0.500000

[2112 rows x 4 columns]

In [266]:
non_key_to_nan("Census_OEMNameIdentifier")

In [268]:
to_categorical("Census_OEMNameIdentifier")

### Census_OEMModelIdentifier

In [272]:
value_counts("Census_OEMModelIdentifier")

train  train_sum  test  train_ratio
1.0           4        1.0     2     0.250000
14.0          1        0.0     1     0.000000
22.0         37       20.0    29     0.540541
23.0         65       34.0    40     0.523077
31.0          2        2.0     4     1.000000
55.0          3        0.0     5     0.000000
56.0          3        1.0     3     0.333333
61.0          1        1.0     4     1.000000
127.0         7        3.0     7     0.428571
128.0         4        2.0     1     0.500000
133.0         2        1.0     2     0.500000
135.0         8        5.0     2     0.625000
136.0         1        0.0     1     0.000000
142.0         6        4.0    21     0.666667
150.0       291      142.0   292     0.487973
152.0        30       12.0    30     0.400000
153.0         7        3.0     8     0.428571
156.0       367      159.0   289     0.433243
165.0         3        1.0     5     0.333333
166.0         5        2.0     6     0.400000
167.0         9        2.0     6     0.222222
168.0         1        0.0     3     0.000000
169.0         1        0.0     4     0.000000
171.0        60       16.0    46     0.266667
172.0       105       33.0    97     0.314286
173.0         3        1.0     2     0.333333
174.0        23        8.0    24     0.347826
175.0        38       12.0    34     0.315789
177.0         4        2.0     4     0.500000
178.0         8        3.0     4     0.375000
...         ...        ...   ...          ...
345324.0      2        2.0     1     1.000000
345326.0      1        1.0     1     1.000000
345356.0      4        1.0     8     0.250000
345357.0     27       14.0    30     0.518519
345361.0      3        0.0     2     0.000000
345381.0      1        0.0     2     0.000000
345382.0      3        2.0     5     0.666667
345385.0      3        1.0     7     0.333333
345386.0     12        6.0    14     0.500000
345387.0      4        0.0     8     0.000000
345391.0      2        1.0     2     0.500000
345400.0      3        0.0     2     0.000000
345401.0      2        1.0     1     0.500000
345410.0      2        1.0     1     0.500000
345428.0      1        0.0     1     0.000000
345433.0      1        1.0     2     1.000000
345463.0      1        0.0     1     0.000000
345464.0     21       12.0    19     0.571429
345470.0      5        5.0     4     1.000000
345473.0     12        3.0     8     0.250000
345475.0      1        1.0     1     1.000000
345476.0      2        1.0     1     0.500000
345478.0      1        0.0     2     0.000000
345479.0      4        0.0     2     0.000000
345480.0     16        9.0    28     0.562500
345485.0      5        2.0    17     0.400000
345489.0      2        1.0     2     0.500000
345490.0     31       15.0    10     0.483871
345494.0      7        5.0     4     0.714286
345498.0      1        0.0     1     0.000000

[109898 rows x 4 columns]

In [271]:
non_key_to_nan("Census_OEMModelIdentifier")

In [273]:
to_categorical("Census_OEMModelIdentifier")

### Census_ProcessorCoreCount

In [274]:
value_counts("Census_ProcessorCoreCount")

train  train_sum       test  train_ratio
1.0      70390.0    20768.0    62056.0     0.295042
2.0    2311969.0  1063217.0  1995987.0     0.459875
3.0      13580.0     6193.0    12313.0     0.456038
4.0    5430193.0  2758076.0  4646102.0     0.507915
5.0        216.0       99.0      197.0     0.458333
6.0      69910.0    39597.0    80310.0     0.566400
7.0         92.0       37.0       78.0     0.402174
8.0     865004.0   480084.0   843530.0     0.555008
9.0          7.0        3.0        4.0     0.428571
10.0        98.0       35.0      108.0     0.357143
11.0         4.0        2.0        3.0     0.500000
12.0     92702.0    54202.0   120673.0     0.584691
13.0         1.0        0.0        1.0     0.000000
14.0        22.0        9.0       27.0     0.409091
15.0         2.0        1.0        4.0     0.500000
16.0     18551.0    10418.0    22954.0     0.561587
18.0         5.0        0.0       15.0     0.000000
19.0         NaN        NaN        1.0          NaN
20.0      1781.0      910.0     1799.0     0.510949
22.0         4.0        0.0        4.0     0.000000
23.0         NaN        NaN        1.0          NaN
24.0      1847.0      930.0     1979.0     0.503519
25.0         1.0        1.0        NaN     1.000000
26.0         2.0        0.0        1.0     0.000000
28.0       271.0      136.0      289.0     0.501845
30.0        10.0        3.0        6.0     0.300000
32.0      2136.0     1065.0     2204.0     0.498596
35.0         NaN        NaN        1.0          NaN
36.0       287.0      127.0      267.0     0.442509
40.0       506.0      240.0      424.0     0.474308
44.0        16.0       10.0       18.0     0.625000
46.0         3.0        3.0        3.0     1.000000
48.0       235.0       97.0      224.0     0.412766
50.0         1.0        0.0        NaN     0.000000
52.0         3.0        0.0        3.0     0.000000
54.0         1.0        0.0        NaN     0.000000
56.0       132.0       56.0      115.0     0.424242
64.0        93.0       39.0      182.0     0.419355
72.0        39.0       13.0       44.0     0.333333
80.0        20.0       12.0       15.0     0.600000
88.0        23.0       13.0       20.0     0.565217
96.0         6.0        2.0        5.0     0.333333
104.0        2.0        0.0        3.0     0.000000
112.0        6.0        0.0        2.0     0.000000
120.0        1.0        1.0        1.0     1.000000
128.0        3.0        1.0        1.0     0.333333
144.0        1.0        0.0        1.0     0.000000
192.0        1.0        1.0        NaN     1.000000
224.0        NaN        NaN        1.0          NaN

### Census_ProcessorManufacturerIdentifier

In [277]:
value_counts("Census_ProcessorManufacturerIdentifier")

train  train_sum     test  train_ratio
1.0   1040292   509719.0   913445     0.489977
3.0       218       51.0      183     0.233945
5.0   7839318  3926620.0  6878134     0.500888
10.0      339        5.0      207     0.014749

In [276]:
non_key_to_nan("Census_ProcessorManufacturerIdentifier")

In [278]:
to_categorical("Census_ProcessorManufacturerIdentifier")

### Census_ProcessorModelIdentifier

In [282]:
value_counts("Census_ProcessorModelIdentifier")

train  train_sum   test  train_ratio
3.0         7        3.0      4     0.428571
6.0         2        0.0      6     0.000000
7.0         3        2.0      2     0.666667
19.0     1005      610.0    708     0.606965
20.0        2        0.0      4     0.000000
23.0      244      129.0    287     0.528689
24.0        8        6.0      5     0.750000
25.0      220      124.0    231     0.563636
26.0        3        1.0      1     0.333333
27.0       92       53.0     91     0.576087
28.0        2        1.0      2     0.500000
29.0     3945     1948.0   3641     0.493790
30.0      354      181.0    163     0.511299
31.0     2001     1107.0   1618     0.553223
32.0      187       91.0    110     0.486631
33.0        1        1.0      2     1.000000
34.0     1156      627.0   1015     0.542388
35.0      140       88.0     80     0.628571
37.0     6147     3221.0   4687     0.523995
38.0      514      280.0    228     0.544747
39.0     3989     2088.0   3664     0.523439
40.0      477      226.0    280     0.473795
41.0      955      524.0    835     0.548691
42.0      116       43.0    104     0.370690
43.0      110       67.0    147     0.609091
44.0       10        6.0      5     0.600000
45.0     1969     1069.0   1787     0.542915
46.0       68       46.0     21     0.676471
48.0     1486      813.0   1277     0.547106
49.0      232      141.0    127     0.607759
...       ...        ...    ...          ...
4336.0  62054    25871.0  51891     0.416911
4344.0      1        0.0      3     0.000000
4348.0      8        2.0      1     0.250000
4360.0     96       28.0     81     0.291667
4364.0      3        2.0      5     0.666667
4368.0     13        6.0     15     0.461538
4372.0      9        2.0      7     0.222222
4376.0     16        4.0      8     0.250000
4380.0      3        0.0      4     0.000000
4384.0      2        1.0      5     0.500000
4388.0      1        0.0      2     0.000000
4392.0      2        0.0      1     0.000000
4400.0      2        1.0      2     0.500000
4404.0   4418     2161.0   4034     0.489135
4408.0    343      146.0    165     0.425656
4416.0     12        7.0      5     0.583333
4420.0     10        6.0      7     0.600000
4424.0     40        4.0     10     0.100000
4428.0    121        0.0     26     0.000000
4432.0    188        5.0    174     0.026596
4436.0     92       27.0     71     0.293478
4440.0     47        8.0     39     0.170213
4448.0     19        7.0     16     0.368421
4452.0     26        7.0     28     0.269231
4456.0      9        1.0     12     0.111111
4460.0     13        1.0      7     0.076923
4464.0      3        0.0      1     0.000000
4468.0    192       51.0    162     0.265625
4472.0    116       14.0    114     0.120690
4476.0      1        1.0      4     1.000000

[2316 rows x 4 columns]

In [280]:
non_key_to_nan("Census_ProcessorModelIdentifier")

In [281]:
to_categorical("Census_ProcessorModelIdentifier")

### Census_ProcessorClass

In [283]:
value_counts("Census_ProcessorClass")

train  train_sum   test  train_ratio
high   6096     3635.0   3073     0.596293
low    9621     4816.0   4682     0.500572
mid   20914    11566.0  10476     0.553027

### Census_PrimaryDiskTotalCapacity

In [284]:
value_counts("Census_PrimaryDiskTotalCapacity")

train  train_sum  test  train_ratio
0.000000e+00   15.0       13.0   NaN     0.866667
5.120000e+03    1.0        0.0   NaN     0.000000
6.666000e+03    1.0        0.0   NaN     0.000000
7.296000e+03    1.0        0.0   NaN     0.000000
7.326000e+03    1.0        0.0   NaN     0.000000
7.641000e+03    1.0        0.0   4.0     0.000000
8.191000e+03    1.0        0.0   NaN     0.000000
8.192000e+03    2.0        0.0   NaN     0.000000
8.704000e+03    NaN        NaN   1.0          NaN
9.216000e+03    1.0        0.0   1.0     0.000000
9.825000e+03    NaN        NaN   1.0          NaN
9.952000e+03    NaN        NaN   1.0          NaN
9.983000e+03    NaN        NaN   1.0          NaN
1.011500e+04    1.0        0.0   NaN     0.000000
1.022700e+04    1.0        1.0   NaN     1.000000
1.024000e+04   41.0        7.0  33.0     0.170732
1.024500e+04    1.0        0.0   NaN     0.000000
1.024600e+04    2.0        0.0   1.0     0.000000
1.028100e+04    1.0        0.0   2.0     0.000000
1.032800e+04    1.0        1.0   NaN     1.000000
1.035400e+04    1.0        0.0   NaN     0.000000
1.037000e+04    NaN        NaN   3.0          NaN
1.042600e+04    NaN        NaN   1.0          NaN
1.046900e+04    1.0        1.0   NaN     1.000000
1.047100e+04    1.0        0.0   NaN     0.000000
1.049500e+04    NaN        NaN   1.0          NaN
1.049600e+04    1.0        0.0   NaN     0.000000
1.053400e+04    NaN        NaN   1.0          NaN
1.056600e+04    1.0        0.0   NaN     0.000000
1.066000e+04    NaN        NaN   1.0          NaN
...             ...        ...   ...          ...
2.670790e+07    1.0        1.0   NaN     1.000000
2.859591e+07    1.0        0.0   NaN     0.000000
2.861023e+07    NaN        NaN   1.0          NaN
2.861312e+07    1.0        1.0   NaN     1.000000
3.221225e+07    NaN        NaN   1.0          NaN
3.433227e+07    1.0        0.0   NaN     0.000000
3.433880e+07    2.0        0.0   NaN     0.000000
4.000000e+07    NaN        NaN   1.0          NaN
4.005837e+07    NaN        NaN   1.0          NaN
4.578202e+07    NaN        NaN   2.0          NaN
4.578221e+07    1.0        0.0   NaN     0.000000
4.578507e+07    1.0        0.0   NaN     0.000000
4.768767e+07    1.0        0.0   NaN     0.000000
5.120000e+07    1.0        0.0   NaN     0.000000
5.150821e+07    1.0        0.0   NaN     0.000000
5.341258e+07    NaN        NaN   1.0          NaN
5.722214e+07    NaN        NaN   1.0          NaN
6.104269e+07    NaN        NaN   1.0          NaN
6.451200e+07    NaN        NaN   1.0          NaN
6.710374e+07    NaN        NaN   1.0          NaN
6.710886e+07    2.0        1.0   1.0     0.500000
6.867302e+07    1.0        1.0   NaN     1.000000
7.439411e+07    NaN        NaN   1.0          NaN
7.500000e+07    NaN        NaN   1.0          NaN
9.537024e+07    1.0        0.0   NaN     0.000000
9.537504e+07    NaN        NaN   1.0          NaN
1.068252e+08    NaN        NaN   1.0          NaN
6.442405e+12    NaN        NaN   1.0          NaN
6.523912e+12    1.0        1.0   NaN     1.000000
8.160437e+12    2.0        1.0   NaN     0.500000

[8796 rows x 4 columns]

### Census_PrimaryDiskTypeName

In [285]:
value_counts("Census_PrimaryDiskTypeName")

train  train_sum     test  train_ratio
Census_PrimaryDiskTypeName                                          
HDD                         5806804  2941049.0  4894312     0.506483
SSD                         2466808  1226622.0  2360847     0.497251
UNKNOWN                      358251   161091.0   385419     0.449660
Unspecified                  276776   123941.0   205868     0.447803

### Census_SystemVolumeTotalCapacity

In [286]:
value_counts("Census_SystemVolumeTotalCapacity")

train  train_sum  test  train_ratio
0.0          23.0       22.0   4.0     0.956522
5086.0        1.0        0.0   NaN     0.000000
6500.0        NaN        NaN   1.0          NaN
6658.0        1.0        0.0   NaN     0.000000
6679.0        1.0        0.0   NaN     0.000000
6775.0        1.0        0.0   NaN     0.000000
7000.0        1.0        0.0   NaN     0.000000
7090.0        NaN        NaN   4.0          NaN
7227.0        1.0        0.0   NaN     0.000000
7385.0        1.0        0.0   NaN     0.000000
7451.0        NaN        NaN   1.0          NaN
7639.0        1.0        0.0   NaN     0.000000
7640.0        1.0        0.0   NaN     0.000000
7641.0        1.0        0.0   NaN     0.000000
7667.0        1.0        1.0   NaN     1.000000
7690.0        1.0        0.0   NaN     0.000000
7969.0        NaN        NaN   1.0          NaN
7993.0        1.0        0.0   1.0     0.000000
8000.0        1.0        0.0   NaN     0.000000
8018.0        NaN        NaN   1.0          NaN
8153.0        NaN        NaN   1.0          NaN
8187.0        NaN        NaN   1.0          NaN
8192.0        1.0        0.0   NaN     0.000000
8234.0        NaN        NaN   1.0          NaN
8235.0        1.0        0.0   NaN     0.000000
8451.0        1.0        0.0   NaN     0.000000
8473.0        NaN        NaN   1.0          NaN
8630.0        1.0        0.0   NaN     0.000000
8654.0        1.0        0.0   NaN     0.000000
8665.0        NaN        NaN   1.0          NaN
...           ...        ...   ...          ...
11445630.0    NaN        NaN   1.0          NaN
11445653.0    1.0        0.0   NaN     0.000000
11445678.0    1.0        0.0   NaN     0.000000
11445712.0    1.0        1.0   1.0     1.000000
11445719.0    1.0        0.0   NaN     0.000000
11445953.0    1.0        0.0   NaN     0.000000
11446016.0    1.0        1.0   NaN     1.000000
11446039.0    1.0        0.0   NaN     0.000000
12577438.0    1.0        0.0   NaN     0.000000
13349833.0    1.0        0.0   NaN     0.000000
14335433.0    1.0        0.0   NaN     0.000000
15256974.0    NaN        NaN   1.0          NaN
15259081.0    1.0        0.0   NaN     0.000000
15260120.0    1.0        0.0   NaN     0.000000
15261092.0    NaN        NaN   1.0          NaN
17168788.0    1.0        0.0   NaN     0.000000
17169172.0    1.0        0.0   NaN     0.000000
19063372.0    1.0        1.0   NaN     1.000000
20970952.0    NaN        NaN   1.0          NaN
22892032.0    1.0        1.0   NaN     1.000000
22892040.0    1.0        1.0   NaN     1.000000
26703606.0    1.0        1.0   NaN     1.000000
26707676.0    1.0        1.0   NaN     1.000000
28595580.0    1.0        0.0   NaN     0.000000
34338236.0    1.0        0.0   NaN     0.000000
37000004.0    NaN        NaN   1.0          NaN
45781448.0    NaN        NaN   1.0          NaN
47687100.0    1.0        0.0   NaN     0.000000
57221576.0    NaN        NaN   1.0          NaN
95374808.0    NaN        NaN   1.0          NaN

[636117 rows x 4 columns]

### Census_HasOpticalDiskDrive

In [287]:
value_counts("Census_HasOpticalDiskDrive")

train  train_sum     test  train_ratio
Census_HasOpticalDiskDrive                                          
0                           8232858  4089910.0  7321932     0.496779
1                            688625   368982.0   531321     0.535824

### Census_TotalPhysicalRAM

In [288]:
value_counts("Census_TotalPhysicalRAM")

train  train_sum   test  train_ratio
255.0        1.0        0.0    NaN     0.000000
256.0        NaN        NaN    1.0          NaN
400.0        2.0        0.0    NaN     0.000000
495.0        NaN        NaN    1.0          NaN
503.0        1.0        0.0    NaN     0.000000
511.0        1.0        1.0    3.0     1.000000
512.0      107.0       16.0   84.0     0.149533
594.0        NaN        NaN    1.0          NaN
629.0        1.0        0.0    NaN     0.000000
640.0        2.0        0.0    NaN     0.000000
652.0        NaN        NaN    1.0          NaN
700.0        4.0        0.0    NaN     0.000000
724.0        NaN        NaN    1.0          NaN
762.0        1.0        0.0    NaN     0.000000
767.0        1.0        0.0    1.0     0.000000
768.0       32.0        9.0   24.0     0.281250
784.0        1.0        1.0    NaN     1.000000
799.0        NaN        NaN    1.0          NaN
800.0        2.0        0.0    NaN     0.000000
829.0        2.0        0.0    1.0     0.000000
844.0        1.0        0.0    NaN     0.000000
856.0        NaN        NaN    1.0          NaN
859.0        NaN        NaN    1.0          NaN
879.0        NaN        NaN    1.0          NaN
894.0        2.0        0.0    3.0     0.000000
895.0        1.0        0.0    NaN     0.000000
896.0        2.0        1.0    NaN     0.500000
900.0        1.0        0.0    1.0     0.000000
909.0        NaN        NaN    1.0          NaN
911.0        1.0        0.0    NaN     0.000000
...          ...        ...    ...          ...
153599.0     3.0        3.0    3.0     1.000000
155648.0     1.0        0.0    NaN     0.000000
163840.0    13.0        4.0    5.0     0.307692
167936.0     1.0        0.0    NaN     0.000000
168959.0     1.0        0.0    NaN     0.000000
180224.0     1.0        0.0    2.0     0.000000
196608.0    51.0       21.0   57.0     0.411765
200704.0     NaN        NaN    1.0          NaN
204800.0     1.0        1.0    1.0     1.000000
212992.0     1.0        0.0    NaN     0.000000
229376.0     5.0        1.0    3.0     0.200000
245760.0     4.0        3.0    2.0     0.750000
261119.0     1.0        1.0    NaN     1.000000
262108.0     1.0        0.0    NaN     0.000000
262144.0   117.0       48.0  101.0     0.410256
278528.0     1.0        1.0    NaN     1.000000
294912.0     1.0        0.0    3.0     0.000000
311296.0     1.0        0.0    NaN     0.000000
327680.0     1.0        0.0    3.0     0.000000
360448.0     1.0        0.0    1.0     0.000000
393216.0    15.0        3.0   15.0     0.200000
458752.0     2.0        1.0    NaN     0.500000
524288.0    19.0        7.0   22.0     0.368421
557056.0     1.0        0.0    NaN     0.000000
671744.0     1.0        0.0    NaN     0.000000
786432.0     5.0        1.0    1.0     0.200000
917504.0     1.0        0.0    NaN     0.000000
1048576.0    4.0        0.0    3.0     0.000000
1572864.0    1.0        0.0    NaN     0.000000
6238464.0    NaN        NaN    1.0          NaN

[5276 rows x 4 columns]

### Census_ChassisTypeName

In [292]:
value_counts("Census_ChassisTypeName")

train  train_sum     test  train_ratio
0                        133       53.0       93     0.398496
127                        7        1.0        3     0.142857
25                         6        3.0        1     0.500000
30                       243       78.0       54     0.320988
31                        11        6.0        3     0.545455
32                         8        5.0        2     0.625000
35                        50       16.0        9     0.320000
36                         3        2.0        2     0.666667
44                         4        1.0        1     0.250000
82                         1        1.0        1     1.000000
88                         8        2.0        3     0.250000
AllinOne              204295   105402.0   176207     0.515930
Blade                     52       13.0       17     0.250000
BladeEnclosure             2        1.0        2     0.500000
BusExpansionChassis      720      249.0      681     0.345833
CompactPCI                 2        0.0        1     0.000000
Convertible            84472    40399.0    78658     0.478253
Desktop              1872125   982113.0  1777669     0.524598
Detachable             51466    17011.0    42527     0.330529
DockingStation             2        0.0       13     0.000000
ExpansionChassis          12        5.0        9     0.416667
HandHeld               46009    13456.0    36619     0.292465
Laptop                685581   337884.0   601196     0.492843
LowProfileDesktop      50072    25386.0    46903     0.506990
LunchBox                3971     1744.0     3841     0.439184
MainServerChassis       9545     3772.0     8557     0.395181
MiniPC                  4433     1633.0     4327     0.368374
MiniTower              85127    45022.0    81411     0.528880
MultisystemChassis        61       17.0       41     0.278689
Notebook             5248812  2629216.0  4498390     0.500916
Other                  75782    20043.0    73580     0.264482
PizzaBox                  46       26.0       16     0.565217
Portable              360903   173541.0   307035     0.480852
RackMountChassis        3410     1179.0     2929     0.345748
SealedCasePC              39       19.0       42     0.487179
SpaceSaving            29070    14330.0    26320     0.492948
StickPC                  142       44.0      167     0.309859
SubChassis                16        9.0       19     0.562500
SubNotebook              807      270.0      813     0.334572
Tablet                 13630     5131.0    11294     0.376449
Tower                  12549     6331.0    11644     0.504502
Unknown                77223    34166.0    61806     0.442433

In [290]:
change_value("Census_ChassisTypeName", "UNKNOWN", "Unknown")

In [291]:
non_key_to_nan("Census_ChassisTypeName")

### Census_InternalPrimaryDiagonalDisplaySizeInInches

In [293]:
value_counts("Census_InternalPrimaryDiagonalDisplaySizeInInches")

train  train_sum   test  train_ratio
0.700195      2.0        0.0    3.0     0.000000
2.000000      NaN        NaN    1.0          NaN
2.099609      NaN        NaN    1.0          NaN
3.000000      5.0        2.0    9.0     0.400000
3.199219      1.0        0.0    2.0     0.000000
3.300781      1.0        0.0    NaN     0.000000
3.500000      2.0        0.0    1.0     0.000000
3.900391      5.0        2.0    3.0     0.400000
4.101562      NaN        NaN    2.0          NaN
4.199219      1.0        1.0    1.0     1.000000
4.300781      NaN        NaN    1.0          NaN
4.500000      3.0        1.0    NaN     0.333333
4.699219      2.0        0.0    5.0     0.000000
4.800781      1.0        0.0    1.0     0.000000
4.898438     41.0        9.0   19.0     0.219512
5.000000     15.0        3.0   10.0     0.200000
5.101562      3.0        0.0    2.0     0.000000
5.199219      1.0        1.0    2.0     1.000000
5.300781      3.0        0.0    2.0     0.000000
5.398438     15.0        0.0    3.0     0.000000
5.500000    422.0      134.0  331.0     0.317536
5.601562      5.0        2.0    NaN     0.400000
5.699219      1.0        0.0    NaN     0.000000
5.800781     48.0        5.0   31.0     0.104167
5.898438      2.0        1.0    3.0     0.500000
6.000000    105.0       45.0   86.0     0.428571
6.101562      3.0        1.0    5.0     0.333333
6.199219     14.0        6.0   10.0     0.428571
6.300781      NaN        NaN    1.0          NaN
6.398438      6.0        2.0    4.0     0.333333
...           ...        ...    ...          ...
111.500000    NaN        NaN    1.0          NaN
112.187500    1.0        0.0    NaN     0.000000
112.500000    NaN        NaN    1.0          NaN
113.312500    NaN        NaN    2.0          NaN
114.187500    NaN        NaN    1.0          NaN
115.125000    3.0        0.0    5.0     0.000000
115.625000   14.0        8.0   12.0     0.571429
116.687500   18.0        6.0    9.0     0.333333
118.625000    1.0        1.0    NaN     1.000000
118.875000    1.0        0.0    2.0     0.000000
120.500000    1.0        0.0    NaN     0.000000
122.125000    2.0        0.0    3.0     0.000000
122.687500    NaN        NaN    1.0          NaN
125.500000    NaN        NaN    2.0          NaN
126.812500    NaN        NaN    1.0          NaN
127.375000    NaN        NaN    1.0          NaN
128.125000    NaN        NaN    2.0          NaN
129.625000    9.0        5.0    6.0     0.555556
131.250000    1.0        1.0    NaN     1.000000
133.750000    1.0        0.0    NaN     0.000000
135.125000    1.0        1.0    NaN     1.000000
139.375000    1.0        1.0    NaN     1.000000
141.875000    NaN        NaN    2.0          NaN
142.000000  136.0       27.0  114.0     0.198529
142.500000    NaN        NaN    1.0          NaN
144.500000    NaN        NaN    1.0          NaN
173.500000    9.0        0.0    4.0     0.000000
182.250000    1.0        0.0    NaN     0.000000
202.500000    NaN        NaN    1.0          NaN
213.750000    NaN        NaN    1.0          NaN

[869 rows x 4 columns]

### Census_InternalPrimaryDisplayResolutionHorizontal

In [294]:
value_counts("Census_InternalPrimaryDisplayResolutionHorizontal")

train  train_sum  test  train_ratio
-1.0      156.0       44.0  99.0     0.282051
 140.0      NaN        NaN   1.0          NaN
 144.0      1.0        0.0   NaN     0.000000
 200.0      5.0        2.0   9.0     0.400000
 240.0      NaN        NaN   1.0          NaN
 274.0      1.0        0.0   NaN     0.000000
 320.0     44.0       20.0  28.0     0.454545
 325.0      1.0        1.0   NaN     1.000000
 328.0      NaN        NaN   1.0          NaN
 331.0      1.0        1.0   NaN     1.000000
 333.0      1.0        0.0   NaN     0.000000
 360.0      1.0        0.0   1.0     0.000000
 361.0      NaN        NaN   1.0          NaN
 384.0      NaN        NaN   3.0          NaN
 392.0      NaN        NaN   1.0          NaN
 396.0      NaN        NaN   1.0          NaN
 400.0      3.0        2.0   3.0     0.666667
 432.0      2.0        1.0   NaN     0.500000
 433.0      NaN        NaN   1.0          NaN
 474.0      NaN        NaN   1.0          NaN
 479.0      NaN        NaN   1.0          NaN
 480.0     85.0       20.0  63.0     0.235294
 491.0      NaN        NaN   1.0          NaN
 499.0      NaN        NaN   1.0          NaN
 510.0      1.0        1.0   NaN     1.000000
 512.0     13.0        6.0  11.0     0.461538
 519.0      NaN        NaN   1.0          NaN
 525.0      NaN        NaN   1.0          NaN
 533.0      NaN        NaN   1.0          NaN
 536.0      NaN        NaN   1.0          NaN
...         ...        ...   ...          ...
 6048.0     3.0        2.0   2.0     0.666667
 6064.0     1.0        0.0   NaN     0.000000
 6080.0     1.0        1.0   NaN     1.000000
 6092.0     NaN        NaN   1.0          NaN
 6144.0     1.0        0.0   1.0     0.000000
 6160.0     1.0        1.0   NaN     1.000000
 6236.0     1.0        0.0   NaN     0.000000
 6304.0     NaN        NaN   1.0          NaN
 6400.0     9.0        3.0  14.0     0.333333
 6464.0     NaN        NaN   1.0          NaN
 6480.0     4.0        0.0   1.0     0.000000
 6720.0     3.0        1.0   3.0     0.333333
 6880.0     4.0        1.0   4.0     0.250000
 7200.0     1.0        1.0   NaN     1.000000
 7560.0     NaN        NaN   1.0          NaN
 7680.0    40.0       17.0  46.0     0.425000
 7688.0     NaN        NaN   1.0          NaN
 7756.0     NaN        NaN   1.0          NaN
 7808.0     NaN        NaN   1.0          NaN
 8000.0     NaN        NaN   1.0          NaN
 8192.0     3.0        2.0   2.0     0.666667
 8680.0     1.0        1.0   NaN     1.000000
 9600.0     2.0        1.0   1.0     0.500000
 10240.0    NaN        NaN   1.0          NaN
 10320.0    4.0        0.0   2.0     0.000000
 10800.0    NaN        NaN   1.0          NaN
 11120.0    NaN        NaN   1.0          NaN
 11520.0    7.0        0.0   6.0     0.000000
 12288.0    3.0        2.0   2.0     0.666667
 15360.0    NaN        NaN   1.0          NaN

[2440 rows x 4 columns]

### Census_InternalPrimaryDisplayResolutionVertical

In [295]:
value_counts("Census_InternalPrimaryDisplayResolutionVertical")

train  train_sum   test  train_ratio
-1.0     156.0       44.0   99.0     0.282051
 140.0     NaN        NaN    1.0          NaN
 171.0     1.0        0.0    NaN     0.000000
 200.0    24.0       11.0   21.0     0.458333
 208.0     1.0        0.0    NaN     0.000000
 228.0     NaN        NaN    1.0          NaN
 240.0    23.0       11.0   18.0     0.478261
 256.0     2.0        1.0    1.0     0.500000
 285.0     1.0        0.0    NaN     0.000000
 287.0     NaN        NaN    1.0          NaN
 300.0     2.0        1.0    2.0     0.500000
 320.0     NaN        NaN    1.0          NaN
 322.0     NaN        NaN    1.0          NaN
 330.0     2.0        1.0    NaN     0.500000
 350.0    23.0       10.0   28.0     0.434783
 358.0     1.0        0.0    NaN     0.000000
 360.0     9.0        0.0    7.0     0.000000
 373.0     1.0        1.0    NaN     1.000000
 375.0     NaN        NaN    1.0          NaN
 383.0     2.0        0.0    NaN     0.000000
 384.0    12.0        6.0   11.0     0.500000
 389.0     NaN        NaN    2.0          NaN
 390.0     1.0        1.0    NaN     1.000000
 393.0     NaN        NaN    1.0          NaN
 394.0     1.0        0.0    NaN     0.000000
 395.0     1.0        0.0    NaN     0.000000
 398.0     1.0        1.0    NaN     1.000000
 400.0   117.0       64.0  115.0     0.547009
 402.0     1.0        0.0    NaN     0.000000
 403.0     NaN        NaN    1.0          NaN
...        ...        ...    ...          ...
 3270.0    1.0        0.0    NaN     0.000000
 3308.0    NaN        NaN    1.0          NaN
 3314.0    1.0        0.0    NaN     0.000000
 3332.0    1.0        0.0    NaN     0.000000
 3338.0    1.0        0.0    NaN     0.000000
 3350.0    NaN        NaN    1.0          NaN
 3366.0    NaN        NaN    2.0          NaN
 3372.0    NaN        NaN    1.0          NaN
 3384.0   10.0        1.0    2.0     0.100000
 3440.0    2.0        1.0    NaN     0.500000
 3478.0    NaN        NaN    3.0          NaN
 3600.0    8.0        2.0   10.0     0.250000
 3648.0    1.0        0.0    NaN     0.000000
 3658.0    1.0        0.0    NaN     0.000000
 3718.0    1.0        0.0    NaN     0.000000
 3736.0    1.0        0.0    NaN     0.000000
 3774.0    1.0        1.0    NaN     1.000000
 3780.0    1.0        1.0    3.0     1.000000
 3840.0  132.0       41.0  112.0     0.310606
 4000.0    NaN        NaN    1.0          NaN
 4050.0    1.0        1.0    NaN     1.000000
 4052.0    NaN        NaN    1.0          NaN
 4200.0    NaN        NaN    1.0          NaN
 4320.0   14.0        4.0   10.0     0.285714
 4500.0    1.0        1.0    1.0     1.000000
 5432.0    1.0        0.0    NaN     0.000000
 5760.0    1.0        0.0    NaN     0.000000
 6480.0    1.0        0.0    NaN     0.000000
 7560.0    NaN        NaN    1.0          NaN
 8640.0    1.0        0.0    NaN     0.000000

[1786 rows x 4 columns]

### Census_PowerPlatformRoleName

In [296]:
value_counts("Census_PowerPlatformRoleName")

train  train_sum     test  train_ratio
AppliancePC           4015     1004.0     3175     0.250062
Desktop            2066620  1083709.0  1962019     0.524387
EnterpriseServer      7094     2823.0     6523     0.397942
Mobile             6182908  3104529.0  5323917     0.502115
PerformanceServer       97       41.0       75     0.422680
SOHOServer           37841    18311.0    34793     0.483893
Slate               492537   181293.0   402960     0.368080
UNKNOWN              20628     9965.0    18482     0.483081
Unspecified              5        3.0       14     0.600000
Workstation         109683    57188.0   101269     0.521393

### Census_InternalBatteryType

In [390]:
value_counts("Census_InternalBatteryType")

train  train_sum     test  train_ratio
lio         4        3.0        5     0.750000
#       183998    95183.0   175307     0.517305
#TAB#        1        1.0        2     1.000000
0ts0         1        1.0        3     1.000000
0x0b         3        2.0        2     0.666667
3500         1        0.0        1     0.000000
4cel       170       62.0       92     0.364706
4lio         4        3.0        5     0.750000
5nm1         2        2.0        4     1.000000
6ion         1        0.0        1     0.000000
a130         1        1.0        1     1.000000
a132        10        3.0        7     0.300000
a138         2        0.0        2     0.000000
a140         2        1.0        1     0.500000
bad         33       20.0       25     0.606061
batt        60       12.0       65     0.200000
bq20      2302      612.0     1918     0.265856
ca48        16        5.0       13     0.312500
h00j         2        1.0        3     0.500000
icp3         3        3.0        5     1.000000
ithi        79       20.0       59     0.253165
lai0         8        4.0        1     0.500000
lgi0       399      239.0      236     0.598997
lgl0         1        1.0        1     1.000000
lgs0         3        1.0        1     0.333333
lhp0       182      104.0       91     0.571429
li        6708     2998.0     4840     0.446929
li p      8383     4190.0     6300     0.499821
li-i    245617   119485.0   174765     0.486469
li-l         1        1.0        1     1.000000
li-p         4        3.0        4     0.750000
liio     32635    15860.0    24624     0.485981
lio          5        2.0        3     0.400000
lion   2028256   979374.0  1428882     0.482865
lip      62099    32999.0    47080     0.531393
lipo       198       75.0      159     0.378788
lipp        83       42.0       44     0.506024
lit         16        3.0       11     0.187500
liÿÿ         1        0.0        1     0.000000
nimh      4614     1927.0     3428     0.417642
ots0         9        3.0        6     0.333333
pad0        22       12.0       17     0.545455
pbac      2274     1294.0     1621     0.569041
ram         35       12.0       31     0.342857
real      2744     1522.0     2115     0.554665
unkn       533      268.0      395     0.502814
vbox      1454      229.0     1176     0.157497
virt        33        3.0       36     0.090909
ÿÿÿÿ         8        6.0        3     0.750000

In [298]:
non_key_to_nan("Census_InternalBatteryType")

In [418]:
remove_columns("Census_InternalBatteryType")

### Census_InternalBatteryNumberOfCharges

In [300]:
value_counts("Census_InternalBatteryNumberOfCharges")

train  train_sum       test  train_ratio
0.000000e+00  5053404.0  2511817.0  4335051.0     0.497054
1.000000e+00    53810.0    22607.0    49750.0     0.420126
2.000000e+00    28128.0    11673.0    26035.0     0.414996
3.000000e+00    21537.0     8957.0    19993.0     0.415889
4.000000e+00    18020.0     7405.0    16654.0     0.410932
5.000000e+00    16190.0     6645.0    14412.0     0.410439
6.000000e+00    14472.0     5971.0    13127.0     0.412590
7.000000e+00    13506.0     5493.0    12218.0     0.406708
8.000000e+00    12985.0     5376.0    11508.0     0.414016
9.000000e+00    12020.0     5090.0    10858.0     0.423461
1.000000e+01    11518.0     4818.0    10183.0     0.418302
1.100000e+01    11082.0     4739.0     9629.0     0.427630
1.200000e+01    10561.0     4494.0     9245.0     0.425528
1.300000e+01     9965.0     4300.0     8816.0     0.431510
1.400000e+01     9803.0     4212.0     8543.0     0.429664
1.500000e+01     9621.0     4140.0     8161.0     0.430309
1.600000e+01    27348.0    12456.0    22695.0     0.455463
1.700000e+01     8937.0     3866.0     7751.0     0.432584
1.800000e+01     8818.0     3804.0     7438.0     0.431390
1.900000e+01     8473.0     3732.0     7360.0     0.440458
2.000000e+01     8322.0     3658.0     6993.0     0.439558
2.100000e+01     8194.0     3604.0     6908.0     0.439834
2.200000e+01     7973.0     3535.0     6610.0     0.443371
2.300000e+01     7710.0     3429.0     6451.0     0.444747
2.400000e+01     7414.0     3318.0     6410.0     0.447532
2.500000e+01     7647.0     3395.0     6161.0     0.443965
2.600000e+01     9243.0     4314.0     8075.0     0.466732
2.700000e+01     7106.0     3167.0     5879.0     0.445680
2.800000e+01     6909.0     3043.0     5846.0     0.440440
2.900000e+01     6812.0     3110.0     5610.0     0.456547
...                 ...        ...        ...          ...
6.544700e+04        NaN        NaN        1.0          NaN
6.545100e+04        1.0        1.0        NaN     1.000000
6.545600e+04        NaN        NaN        1.0          NaN
6.545800e+04        NaN        NaN        1.0          NaN
6.545900e+04        NaN        NaN        1.0          NaN
6.546600e+04        2.0        1.0        NaN     0.500000
6.547700e+04        2.0        1.0        NaN     0.500000
6.548200e+04        NaN        NaN        1.0          NaN
6.548800e+04        1.0        1.0        NaN     1.000000
6.549000e+04        NaN        NaN        1.0          NaN
6.549400e+04        1.0        0.0        NaN     0.000000
6.549500e+04        1.0        1.0        NaN     1.000000
6.549700e+04        1.0        1.0        NaN     1.000000
6.549900e+04        1.0        0.0        NaN     0.000000
6.550200e+04        1.0        0.0        NaN     0.000000
6.550400e+04        1.0        1.0        NaN     1.000000
6.550700e+04        NaN        NaN        1.0          NaN
6.550900e+04        1.0        0.0        NaN     0.000000
6.551200e+04        NaN        NaN        1.0          NaN
6.551300e+04        NaN        NaN        1.0          NaN
6.551600e+04        1.0        0.0        NaN     0.000000
6.551800e+04        NaN        NaN        1.0          NaN
6.551900e+04        1.0        0.0        2.0     0.000000
6.552500e+04        NaN        NaN        1.0          NaN
6.553000e+04        NaN        NaN        1.0          NaN
6.553100e+04        NaN        NaN        1.0          NaN
6.553200e+04        NaN        NaN        1.0          NaN
6.553400e+04        NaN        NaN        1.0          NaN
6.553500e+04      148.0       49.0       91.0     0.331081
4.294967e+09  2263993.0  1167305.0  2153981.0     0.515596

[52836 rows x 4 columns]

### Census_OSVersion

In [303]:
value_counts("Census_OSVersion")

train  train_sum   test  train_ratio
10.0.10240.16384  25834    12114.0  22687     0.468917
10.0.10240.16387     49       18.0     36     0.367347
10.0.10240.16389     36       11.0     31     0.305556
10.0.10240.16393     34       13.0     31     0.382353
10.0.10240.16394     16        8.0     14     0.500000
10.0.10240.16397    700      332.0    569     0.474286
10.0.10240.16399      4        2.0      4     0.500000
10.0.10240.16405   6626     3070.0   6127     0.463326
10.0.10240.16413    919      430.0    831     0.467900
10.0.10240.16425      1        0.0      2     0.000000
10.0.10240.16430    783      356.0    700     0.454662
10.0.10240.16433    166       70.0    164     0.421687
10.0.10240.16445   1003      443.0    939     0.441675
10.0.10240.16463    653      300.0    568     0.459418
10.0.10240.16487   3770     1822.0   3539     0.483289
10.0.10240.16500     28       10.0     28     0.357143
10.0.10240.16520   1105      492.0    954     0.445249
10.0.10240.16549    355      164.0    343     0.461972
10.0.10240.16566     78       32.0     70     0.410256
10.0.10240.16590    232      112.0    191     0.482759
10.0.10240.16601    304      153.0    220     0.503289
10.0.10240.16644    318      154.0    233     0.484277
10.0.10240.16683    259      120.0    210     0.463320
10.0.10240.16725    343      152.0    252     0.443149
10.0.10240.16732     41       11.0     34     0.268293
10.0.10240.16771    300      132.0    214     0.440000
10.0.10240.16854    523      201.0    426     0.384321
10.0.10240.16942    519      228.0    365     0.439306
10.0.10240.17024    628      249.0    446     0.396497
10.0.10240.17071   1417      569.0   1011     0.401553
...                 ...        ...    ...          ...
10.0.17728.1000      21        7.0      1     0.333333
10.0.17730.1000      24        5.0      1     0.208333
10.0.17730.1001       2        0.0      1     0.000000
10.0.17733.1000     524      234.0     35     0.446565
10.0.17735.1000     980      423.0     60     0.431633
10.0.17738.1000    3062     1142.0    213     0.372959
10.0.17741.1000     814      373.0     84     0.458231
10.0.17742.1000       9        0.0      1     0.000000
10.0.17744.1001     703      310.0    151     0.440967
10.0.17744.1003     949      405.0    217     0.426765
10.0.17744.1004     719      347.0    185     0.482615
10.0.17746.1000    1220      564.0    138     0.462295
10.0.17751.1       1006      423.0    224     0.420477
10.0.17754.1       1071      488.0    394     0.455649
10.0.17754.1000      15        0.0      4     0.000000
10.0.17755.1        684      314.0    125     0.459064
10.0.17756.1          1        0.0      1     0.000000
10.0.17758.1       1181      508.0   2053     0.430144
10.0.17758.4        522      239.0    931     0.457854
10.0.17759.1          1        0.0      2     0.000000
10.0.17760.1        590      258.0    384     0.437288
10.0.17763.1       1063      471.0  56351     0.443086
10.0.18204.1001      20        5.0      1     0.250000
10.0.18219.1000      10        5.0      1     0.500000
10.0.18234.1000     233       88.0     53     0.377682
10.0.18237.1000     173       67.0     76     0.387283
10.0.18242.1000     142       65.0    166     0.457746
6.1.7601.23418        1        0.0      2     0.000000
6.1.7601.24241        2        2.0      1     1.000000
6.3.9600.19069        4        2.0      1     0.500000

[365 rows x 4 columns]

In [302]:
non_key_to_nan("Census_OSVersion")

### Census_OSArchitecture

In [304]:
value_counts("Census_OSArchitecture")

train  train_sum     test  train_ratio
amd64  8105885  4144868.0  7174667     0.511341
arm64      346        5.0      211     0.014451
x86     815252   314019.0   678375     0.385180

### Census_OSBranch

In [307]:
value_counts("Census_OSBranch")

train  train_sum     test  train_ratio
rs1_release                 785534   364050.0   603919     0.463443
rs1_release_srvmedia            10        1.0        2     0.100000
rs2_release                 797066   381620.0   626698     0.478781
rs3_release                1237321   575677.0  1127648     0.465261
rs3_release_svc_escrow     1199767   619778.0   496021     0.516582
rs3_release_svc_escrow_im     6181     2574.0     4276     0.416437
rs4_release                4009158  2092879.0  3979144     0.522025
rs5_release                  15324     6579.0   290608     0.429327
rs5_release_sigma               62        1.0        5     0.016129
rs5_release_sigma_dev            2        0.0        1     0.000000
rs_prerelease                 3171     1207.0     9110     0.380637
rs_prerelease_flt             2714     1287.0      242     0.474208
rs_shell                         1        0.0        1     0.000000
th1                          75764    33014.0    67214     0.435748
th1_st1                     195840    99089.0   183721     0.505969
th2_release                 326655   161993.0   240049     0.495915
th2_release_sec             266882   119134.0   224555     0.446392
win7sp1_ldr                      3        2.0       15     0.666667
win7sp1_ldr_escrow               2        1.0        1     0.500000
winblue_ltsb                     3        0.0        7     0.000000
winblue_ltsb_escrow              8        2.0        2     0.250000

In [306]:
non_key_to_nan("Census_OSBranch")

### Census_OSBuildNumber

In [308]:
value_counts("Census_OSBuildNumber")

train  train_sum      test  train_ratio
7600        1.0        0.0       NaN     0.000000
7601        7.0        4.0      20.0     0.571429
9200        6.0        3.0       NaN     0.500000
9600       12.0        3.0      12.0     0.250000
10240  271604.0   132103.0  250935.0     0.486381
10565       5.0        4.0       1.0     0.800000
10576       2.0        1.0       2.0     0.500000
10586  593527.0   281120.0  464599.0     0.473643
11082       5.0        3.0       1.0     0.600000
11099       3.0        1.0       NaN     0.333333
11102       NaN        NaN       2.0          NaN
14206       1.0        1.0       NaN     1.000000
14216       NaN        NaN       1.0          NaN
14218       NaN        NaN       1.0          NaN
14229       NaN        NaN       1.0          NaN
14230       1.0        1.0       NaN     1.000000
14232       NaN        NaN       1.0          NaN
14246       NaN        NaN       1.0          NaN
14251       NaN        NaN       1.0          NaN
14257       2.0        0.0       1.0     0.000000
14267       1.0        1.0       NaN     1.000000
14271       3.0        0.0       NaN     0.000000
14279       3.0        1.0       2.0     0.333333
14282       NaN        NaN       1.0          NaN
14291       2.0        0.0       NaN     0.000000
14294       NaN        NaN       1.0          NaN
14295      18.0       10.0       9.0     0.555556
14298       1.0        1.0       NaN     1.000000
14300       1.0        1.0       NaN     1.000000
14301       NaN        NaN       1.0          NaN
...         ...        ...       ...          ...
18238       1.0        0.0       NaN     0.000000
18240       5.0        0.0       NaN     0.000000
18241       1.0        0.0       NaN     0.000000
18242     142.0       65.0     166.0     0.457746
18244       1.0        0.0       NaN     0.000000
18247       NaN        NaN     257.0          NaN
18248       NaN        NaN       1.0          NaN
18249       NaN        NaN       3.0          NaN
18252       NaN        NaN    3299.0          NaN
18253       NaN        NaN       3.0          NaN
18255       NaN        NaN       4.0          NaN
18256       NaN        NaN       1.0          NaN
18259       NaN        NaN       8.0          NaN
18260       NaN        NaN       5.0          NaN
18262       NaN        NaN    1197.0          NaN
18264       NaN        NaN       2.0          NaN
18265       NaN        NaN       2.0          NaN
18266       NaN        NaN       1.0          NaN
18267       NaN        NaN     459.0          NaN
18269       NaN        NaN       1.0          NaN
18270       NaN        NaN       2.0          NaN
18272       NaN        NaN     797.0          NaN
18277       NaN        NaN     508.0          NaN
18279       NaN        NaN       3.0          NaN
18280       NaN        NaN       1.0          NaN
18282       NaN        NaN    1214.0          NaN
18285       NaN        NaN       2.0          NaN
18286       NaN        NaN       1.0          NaN
18287       NaN        NaN       2.0          NaN
18289       NaN        NaN       3.0          NaN

[216 rows x 4 columns]

### Census_OSBuildRevision

In [309]:
value_counts("Census_OSBuildRevision")

train  train_sum      test  train_ratio
0      166369.0    85650.0  166919.0     0.514819
1      106583.0    55303.0  179005.0     0.518873
2           NaN        NaN       3.0          NaN
3        6741.0     3588.0    6847.0     0.532265
4         522.0      239.0     931.0     0.457854
5        7580.0     3125.0    5611.0     0.412269
10        940.0      438.0    1240.0     0.465957
11       1723.0      840.0    1920.0     0.487522
13       1056.0      494.0     875.0     0.467803
14       7581.0     3583.0    5932.0     0.472629
15     117555.0    62204.0  123264.0     0.529148
17       2969.0     1501.0    2276.0     0.505557
19       9414.0     4937.0   11097.0     0.524432
29       4460.0     2042.0    2719.0     0.457848
36      11662.0     5677.0    7254.0     0.486795
48      66265.0    31457.0   47862.0     0.474715
51        878.0      441.0     782.0     0.502278
53          NaN        NaN       3.0          NaN
55          NaN        NaN  117337.0          NaN
63       5732.0     2749.0    3820.0     0.479588
64      14039.0     6992.0   13232.0     0.498041
67        367.0      218.0     453.0     0.594005
71       4247.0     2120.0    2499.0     0.499176
73         56.0       24.0      47.0     0.428571
79         90.0       45.0      78.0     0.500000
81      55384.0    25674.0   42187.0     0.463563
82       2283.0      973.0    1579.0     0.426194
83        225.0      104.0     350.0     0.462222
91          1.0        0.0       NaN     0.000000
98      17567.0     9873.0   22493.0     0.562020
...         ...        ...       ...          ...
17889    1693.0     1025.0     313.0     0.605434
17914    1887.0     1313.0      85.0     0.695813
17918     532.0      341.0      41.0     0.640977
17943       1.0        0.0       NaN     0.000000
17946    3455.0     2125.0     425.0     0.615051
17974       4.0        0.0       NaN     0.000000
17976     959.0      500.0    1977.0     0.521376
18005       NaN        NaN    3003.0          NaN
18036       NaN        NaN    1251.0          NaN
18202       NaN        NaN       1.0          NaN
18687       1.0        0.0       NaN     0.000000
18756       1.0        0.0       NaN     0.000000
18778       1.0        0.0       NaN     0.000000
18869       NaN        NaN       1.0          NaN
18933       NaN        NaN       1.0          NaN
19069       4.0        2.0       1.0     0.500000
19102       4.0        0.0       NaN     0.000000
19134       NaN        NaN       4.0          NaN
19155       NaN        NaN       2.0          NaN
19182       NaN        NaN       1.0          NaN
21703       1.0        1.0       NaN     1.000000
23418       1.0        0.0       2.0     0.000000
23539       NaN        NaN       1.0          NaN
23934       NaN        NaN       8.0          NaN
24149       1.0        0.0       NaN     0.000000
24214       1.0        1.0       NaN     1.000000
24241       2.0        2.0       1.0     1.000000
24263       NaN        NaN       3.0          NaN
24289       NaN        NaN       1.0          NaN
41736       5.0        4.0       1.0     0.800000

[325 rows x 4 columns]

### Census_OSEdition

In [315]:
value_counts("Census_OSEdition")

train  train_sum     test  train_ratio
Cloud                           6275     2520.0     6104     0.401594
CloudN                             8        3.0        4     0.375000
Core                         3469991  1637162.0  2773165     0.471806
CoreCountrySpecific           166100    83182.0   177496     0.500795
CoreN                           4790     2181.0     4688     0.455324
CoreSingleLanguage           1945461  1029318.0  1596966     0.529087
Education                      40704    21623.0    37910     0.531225
EducationN                       932      428.0      921     0.459227
Enterprise                     35603    18404.0    34283     0.516923
EnterpriseN                      351      189.0      321     0.538462
EnterpriseS                    20020    10321.0    20000     0.515534
EnterpriseSN                     878      538.0      604     0.612756
Home                               4        2.0        4     0.500000
HomePremium                        2        1.0        1     0.500000
Professional                 3130568  1601860.0  3101958     0.511684
ProfessionalCountrySpecific        5        2.0        3     0.400000
ProfessionalEducation          56698    31083.0    54834     0.548220
ProfessionalEducationN           192      100.0      242     0.520833
ProfessionalN                  28341    14891.0    32466     0.525423
ProfessionalSingleLanguage         3        2.0        2     0.666667
ProfessionalWorkstation          128       41.0      175     0.320312
ProfessionalWorkstationN          13        7.0        8     0.538462
ServerDatacenter                  15        4.0       23     0.266667
ServerDatacenterEval             829      229.0      573     0.276236
ServerRdsh                         4        1.0       24     0.250000
ServerSolution                   683      288.0      620     0.421669
ServerStandard                 10128     3758.0     8050     0.371051
ServerStandardEval              2751      751.0     1787     0.272992
Ultimate                           4        3.0       13     0.750000

In [313]:
change_value("Census_OSEdition", "PRO", "Pro")
change_value("Census_OSEdition", "HomeBasic", "Home")
change_value("Census_OSEdition", "professional", "Professional")
change_value("Census_OSEdition", "Pro", "Professional")
change_value("Census_OSEdition", "Window 10 Enterprise", "Enterprise")

In [314]:
non_key_to_nan("Census_OSEdition")

### Census_OSSkuName

In [318]:
value_counts("Census_OSSkuName")

train  train_sum     test  train_ratio
CLOUD                            6167     2466.0     6024     0.399870
CLOUDN                              7        3.0        4     0.428571
CORE                          3469869  1637109.0  2773064     0.471807
CORE_COUNTRYSPECIFIC           165886    83054.0   177269     0.500669
CORE_N                           4787     2181.0     4689     0.455609
CORE_SINGLELANGUAGE           1945133  1029117.0  1596754     0.529073
DATACENTER_EVALUATION_SERVER      829      229.0      569     0.276236
DATACENTER_SERVER                  14        4.0       27     0.285714
EDUCATION                       40827    21703.0    38152     0.531584
EDUCATION_N                       927      424.0      920     0.457389
ENTERPRISE                      35602    18390.0    34135     0.516544
ENTERPRISEG                         1        0.0        1     0.000000
ENTERPRISE_N                      356      190.0      332     0.533708
ENTERPRISE_S                    20022    10321.0    20003     0.515483
ENTERPRISE_S_N                    881      540.0      603     0.612940
PROFESSIONAL                  3187913  1633312.0  3157300     0.512345
PROFESSIONAL_N                  28522    14983.0    32693     0.525314
PRO_CHINA                           5        2.0        2     0.400000
PRO_FOR_EDUCATION                   3        2.0       20     0.666667
PRO_SINGLE_LANGUAGE                 2        2.0        2     1.000000
PRO_WORKSTATION                   124       39.0      171     0.314516
PRO_WORKSTATION_N                  12        7.0        8     0.583333
SB_SOLUTION_SERVER                684      289.0      620     0.422515
SERVERRDSH                          4        1.0       24     0.250000
STANDARD_EVALUATION_SERVER       2755      751.0     1792     0.272595
STANDARD_SERVER                 10128     3761.0     8045     0.371347
ULTIMATE                            4        3.0       14     0.750000
UNDEFINED                           1        1.0        1     1.000000
UNLICENSED                         17        8.0       13     0.470588

In [317]:
non_key_to_nan("Census_OSSkuName")

### Census_OSInstallTypeName

In [319]:
value_counts("Census_OSInstallTypeName")

train  train_sum     test  train_ratio
Clean             69073    35649.0    74920     0.516106
CleanPCRefresh    53609    24751.0    71942     0.461695
IBSClean        1650733   862680.0  2110259     0.522604
Other            840121   403435.0   809406     0.480211
Refresh          205842    93885.0   256050     0.456102
Reset            649201   311696.0   630962     0.480122
UUPUpgrade      2608037  1373135.0  1847191     0.526501
Update          1593308   761228.0  1109763     0.477766
Upgrade         1251559   592433.0   942760     0.473356

### Census_OSInstallLanguageIdentifier

In [320]:
value_counts("Census_OSInstallLanguageIdentifier")

train  train_sum     test  train_ratio
1.0     79777    38958.0    77615     0.488336
2.0     18270    10953.0    17510     0.599507
3.0     72370    32911.0    56638     0.454760
4.0     35407    14468.0    22665     0.408620
5.0    252887   111350.0   190338     0.440315
6.0     50489    23851.0    41246     0.472400
7.0    512753   249160.0   423065     0.485926
8.0   3179262  1612174.0  2805320     0.507091
9.0   1034201   529033.0   828830     0.511538
10.0   366636   197462.0   320350     0.538578
11.0     3219     1279.0     2807     0.397328
12.0    19906     7269.0    16023     0.365166
13.0    10869     5379.0     8419     0.494894
14.0   432503   203234.0   356466     0.469902
15.0    41514    21455.0    33545     0.516814
16.0    15831     7774.0    14075     0.491062
17.0    83445    45279.0    71324     0.542621
18.0   190828    92566.0   164436     0.485076
19.0    84177    36525.0    64222     0.433907
20.0   169059    81218.0   158416     0.480412
21.0     6865     3564.0     5703     0.519155
22.0     3167     1346.0     2678     0.425008
23.0    29496    11479.0    19118     0.389171
24.0   142175    66904.0    98129     0.470575
25.0   132408    63245.0   140231     0.477652
26.0   334766   170360.0   289091     0.508893
27.0   108176    53144.0    87155     0.491273
28.0    12696     5702.0    12201     0.449118
29.0   492267   249820.0   543163     0.507489
30.0    31956    14282.0    25223     0.446927
31.0    22149    11520.0    18348     0.520114
32.0     4559     2299.0     4268     0.504277
33.0    48800    20426.0    32272     0.418566
34.0    10647     5574.0    10409     0.523528
35.0   204832   106313.0   200274     0.519025
36.0    14585     8545.0    17520     0.585876
37.0   403190   208475.0   458146     0.517064
38.0     3737     1573.0     2413     0.420926
39.0   201525   101352.0   154858     0.502925

In [321]:
to_categorical("Census_OSInstallLanguageIdentifier")

### Census_OSUILocaleIdentifier

In [325]:
value_counts("Census_OSUILocaleIdentifier")

train  train_sum     test  train_ratio
1.0          1        0.0        2     0.000000
3.0          2        2.0        1     1.000000
5.0      81845    40101.0    79959     0.489963
6.0         58       32.0      100     0.551724
7.0         46       23.0       35     0.500000
8.0          4        1.0        1     0.250000
9.0          9        7.0       16     0.777778
11.0     18618    11167.0    17802     0.599796
12.0        98       59.0      152     0.602041
13.0         3        1.0        1     0.333333
14.0         1        0.0        1     0.000000
15.0       187       74.0      115     0.395722
16.0         6        3.0       10     0.500000
17.0      2171     1053.0     2070     0.485030
18.0         1        1.0        1     1.000000
19.0        29       12.0       63     0.413793
20.0     72672    33011.0    56801     0.454246
21.0        27       13.0       32     0.481481
24.0     35558    14510.0    22747     0.408066
25.0         5        2.0       12     0.400000
26.0    252863   111208.0   190217     0.439795
27.0        36       15.0       44     0.416667
28.0     50797    23996.0    41521     0.472390
29.0        68       39.0       77     0.573529
30.0    513995   249447.0   424469     0.485310
31.0   3170824  1606877.0  2800526     0.506770
32.0       207       91.0      229     0.439614
33.0         7        3.0        8     0.428571
34.0   1040042   532015.0   833576     0.511532
35.0    377071   203579.0   327934     0.539896
...        ...        ...      ...          ...
130.0    22721    11855.0    18769     0.521764
131.0       52       31.0       75     0.596154
132.0       27       13.0       12     0.481481
133.0        2        1.0        3     0.500000
134.0        2        1.0        3     0.500000
135.0       11        9.0       10     0.818182
136.0     4629     2337.0     4340     0.504861
138.0        4        2.0        3     0.500000
139.0       14        7.0       23     0.500000
140.0    48933    20472.0    32311     0.418368
141.0       20        6.0       15     0.300000
142.0        4        0.0        2     0.000000
143.0        4        2.0        3     0.500000
144.0        1        1.0        2     1.000000
145.0    10779     5642.0    10481     0.523425
146.0       68       29.0       97     0.426471
148.0   206854   107373.0   202707     0.519076
149.0       18       15.0       37     0.833333
150.0        3        2.0        2     0.666667
151.0    14903     8724.0    17850     0.585385
152.0       48       30.0       92     0.625000
153.0        4        3.0        3     0.750000
155.0        1        1.0        1     1.000000
156.0     7726     3730.0     6842     0.482785
157.0       87       46.0       77     0.528736
158.0   408118   211376.0   463741     0.517929
159.0     3917     1651.0     2500     0.421496
160.0   201983   101591.0   155150     0.502968
161.0      164      106.0      236     0.646341
162.0       42       22.0       37     0.523810

[135 rows x 4 columns]

In [323]:
non_key_to_nan("Census_OSUILocaleIdentifier")

In [324]:
to_categorical("Census_OSUILocaleIdentifier")

### Census_OSWUAutoUpdateOptionsName

In [326]:
value_counts("Census_OSWUAutoUpdateOptionsName")

train  train_sum     test  \
AutoInstallAndRebootAtMaintenanceTime   371475   181112.0   213891   
DownloadNotify                           14371     5024.0    10986   
FullAuto                               3954497  2040783.0  4113612   
Notify                                 2034254   967026.0  1194458   
Off                                      26961    12891.0    20388   
UNKNOWN                                2519925  1252056.0  2299918   

                                       train_ratio  
AutoInstallAndRebootAtMaintenanceTime     0.487548  
DownloadNotify                            0.349593  
FullAuto                                  0.516066  
Notify                                    0.475371  
Off                                       0.478135  
UNKNOWN                                   0.496862

### Census_IsPortableOperatingSystem

In [327]:
value_counts("Census_IsPortableOperatingSystem")

train  train_sum     test  train_ratio
Census_IsPortableOperatingSystem                                          
0                                 8916619  4456201.0  7848659     0.499764
1                                    4864     2691.0     4594     0.553248

In [421]:
remove_columns("Census_IsPortableOperatingSystem")

### Census_GenuineStateName

In [328]:
value_counts("Census_GenuineStateName")

train  train_sum     test  train_ratio
INVALID_LICENSE   801692   395483.0   875327     0.493310
IS_GENUINE       7877597  3935480.0  6729893     0.499579
OFFLINE           228366   123740.0   232931     0.541849
TAMPERED               2        1.0        1     0.500000
UNKNOWN            13826     4188.0    15100     0.302908

### Census_ActivationChannel

In [329]:
value_counts("Census_ActivationChannel")

train  train_sum     test  train_ratio
OEM:DM          3413350  1695147.0  2815395     0.496623
OEM:NONSLP       317980   165240.0   261287     0.519655
Retail          4727589  2325719.0  4051234     0.491946
Retail:TB:Eval     3582      982.0     2359     0.274149
Volume:GVLK      450954   268182.0   710466     0.594699
Volume:MAK         8028     3622.0    12512     0.451171

### Census_IsFlightingInternal

In [330]:
value_counts("Census_IsFlightingInternal")

train  train_sum     test  train_ratio
Census_IsFlightingInternal                                          
0.0                         1512703   737583.0  1179278     0.487593
1.0                              21       13.0       13     0.619048

### Census_IsFlightsDisabled

In [331]:
value_counts("Census_IsFlightsDisabled")

train  train_sum     test  train_ratio
Census_IsFlightsDisabled                                          
0.0                       8760872  4377413.0  7727414     0.499655
1.0                            88        5.0       38     0.056818

### Census_FlightRing

In [334]:
value_counts("Census_FlightRing")

train  train_sum     test  train_ratio
Canary          3        1.0        2     0.333333
Disabled     3722     1875.0     3379     0.503761
Invalid         1        1.0        1     1.000000
NOT_SET    287803   143854.0   270817     0.499835
OSG             7        0.0        4     0.000000
RP           9860     4876.0    16909     0.494523
Retail    8355679  4185842.0  7340769     0.500958
Unknown    243438   113261.0   201243     0.465256
WIF         10322     4635.0    10113     0.449041
WIS         10648     4547.0    10015     0.427029

In [333]:
non_key_to_nan("Census_FlightRing")

### Census_ThresholdOptIn

In [335]:
value_counts("Census_ThresholdOptIn")

train  train_sum     test  train_ratio
Census_ThresholdOptIn                                          
0.0                    3253342  1600755.0  2323086     0.492034
1.0                        816      382.0      652     0.468137

### Census_FirmwareManufacturerIdentifier

In [338]:
value_counts("Census_FirmwareManufacturerIdentifier")

train  train_sum  test  train_ratio
3.0         1        1.0     1     1.000000
9.0         7        2.0     4     0.285714
11.0       32        8.0    20     0.250000
13.0        8        4.0     3     0.500000
16.0        2        1.0     4     0.500000
17.0       93       38.0    79     0.408602
19.0        1        0.0     1     0.000000
20.0        1        1.0     2     1.000000
22.0        3        0.0     1     0.000000
24.0        1        0.0     1     0.000000
25.0        1        0.0     1     0.000000
28.0        3        0.0     1     0.000000
31.0        2        2.0     4     1.000000
33.0        3        2.0     1     0.666667
34.0       29       21.0    10     0.724138
37.0        6        3.0     2     0.500000
41.0        1        0.0     1     0.000000
43.0        1        0.0     3     0.000000
44.0        1        0.0     2     0.000000
46.0       19       10.0    42     0.526316
47.0        5        3.0     7     0.600000
59.0        2        1.0     2     0.500000
60.0        6        4.0     7     0.666667
61.0        3        1.0     2     0.333333
62.0        1        1.0     3     1.000000
63.0        2        1.0    17     0.500000
64.0        3        0.0     1     0.000000
65.0        3        2.0     1     0.666667
66.0       10        6.0     5     0.600000
70.0        3        3.0     1     1.000000
...       ...        ...   ...          ...
1026.0      4        2.0     1     0.500000
1028.0      1        1.0     1     1.000000
1030.0   2082      825.0  1988     0.396254
1034.0      1        0.0     2     0.000000
1036.0     21        7.0    17     0.333333
1037.0      3        1.0     7     0.333333
1038.0    151       12.0    65     0.079470
1039.0      1        1.0     2     1.000000
1040.0    551       87.0   424     0.157895
1043.0      1        1.0     1     1.000000
1044.0     14        6.0    20     0.428571
1045.0      3        1.0     3     0.333333
1053.0      3        0.0     4     0.000000
1054.0      2        1.0    14     0.500000
1058.0     12        4.0    11     0.333333
1061.0     18        7.0     9     0.388889
1065.0     10        5.0     3     0.500000
1066.0      1        1.0     1     1.000000
1067.0      9        0.0     7     0.000000
1069.0    222       90.0   261     0.405405
1070.0      5        2.0     5     0.400000
1072.0      5        3.0     5     0.600000
1075.0   4598     1727.0  3791     0.375598
1078.0      1        1.0     1     1.000000
1079.0    108       41.0    61     0.379630
1080.0    842      377.0   572     0.447743
1081.0    194       88.0    94     0.453608
1083.0    173       78.0   148     0.450867
1084.0     20       16.0    16     0.800000
1087.0      7        3.0     6     0.428571

[575 rows x 4 columns]

In [337]:
non_key_to_nan("Census_FirmwareManufacturerIdentifier")

In [339]:
to_categorical("Census_FirmwareManufacturerIdentifier")

### Census_FirmwareVersionIdentifier

In [342]:
value_counts("Census_FirmwareVersionIdentifier")

train  train_sum  test  train_ratio
5.0          1        0.0     1     0.000000
18.0         2        1.0     4     0.500000
19.0         4        1.0     1     0.250000
26.0         1        0.0     1     0.000000
27.0         3        2.0     1     0.666667
29.0         3        1.0     1     0.333333
31.0         2        2.0     2     1.000000
33.0         9        6.0     3     0.666667
34.0         6        3.0     2     0.500000
36.0         4        4.0     3     1.000000
37.0         7        4.0     3     0.571429
38.0        15        6.0     4     0.400000
39.0         9        4.0     2     0.444444
40.0        12        8.0     6     0.666667
42.0        18       12.0     6     0.666667
43.0        15        9.0     6     0.600000
45.0         4        3.0     2     0.750000
46.0         9        6.0     6     0.666667
47.0         3        3.0     1     1.000000
48.0         6        2.0     1     0.333333
49.0         6        3.0     2     0.500000
50.0        18       10.0     4     0.555556
51.0        11        6.0     2     0.545455
53.0         2        1.0     1     0.500000
55.0         2        1.0     2     0.500000
56.0        17       12.0    10     0.705882
61.0         3        1.0     6     0.333333
62.0         2        1.0     5     0.500000
63.0         7        4.0     1     0.571429
64.0        11        5.0     5     0.454545
...        ...        ...   ...          ...
72050.0      1        0.0     2     0.000000
72051.0      3        0.0     4     0.000000
72052.0      1        0.0     1     0.000000
72054.0      3        1.0     3     0.333333
72055.0      3        1.0     4     0.333333
72056.0     16        9.0     8     0.562500
72057.0      6        3.0     6     0.500000
72058.0      1        0.0     4     0.000000
72062.0     44       21.0    24     0.477273
72063.0      3        0.0    10     0.000000
72065.0      1        0.0     1     0.000000
72066.0      1        0.0     2     0.000000
72070.0     21        7.0    15     0.333333
72071.0      3        2.0     3     0.666667
72072.0     17        6.0     9     0.352941
72073.0     27        5.0     9     0.185185
72074.0     65       29.0    31     0.446154
72075.0      8        0.0     7     0.000000
72076.0      5        1.0     4     0.200000
72077.0     44       24.0    34     0.545455
72078.0      9        5.0    12     0.555556
72079.0     19        8.0    19     0.421053
72080.0     86       35.0    98     0.406977
72087.0     10        3.0    11     0.300000
72088.0      6        3.0    11     0.500000
72089.0     14        7.0     5     0.500000
72090.0     15        9.0    11     0.600000
72091.0     50       24.0    39     0.480000
72096.0      4        0.0     5     0.000000
72102.0      3        0.0     1     0.000000

[41348 rows x 4 columns]

In [341]:
non_key_to_nan("Census_FirmwareVersionIdentifier")

In [343]:
to_categorical("Census_FirmwareVersionIdentifier")

### Census_IsSecureBootEnabled

In [344]:
value_counts("Census_IsSecureBootEnabled")

train  train_sum     test  train_ratio
Census_IsSecureBootEnabled                                          
0                           4585438  2295583.0  4308612     0.500625
1                           4336045  2163309.0  3544641     0.498913

### Census_IsWIMBootEnabled

In [345]:
value_counts("Census_IsWIMBootEnabled")

train  train_sum     test  train_ratio
Census_IsWIMBootEnabled                                          
0.0                      3261779  1604572.0  2330545     0.491932
1.0                            1        0.0        1     0.000000

In [348]:
remove_columns(["Census_IsWIMBootEnabled"])

### Census_IsVirtualDevice

In [349]:
value_counts("Census_IsVirtualDevice")

train  train_sum     test  train_ratio
Census_IsVirtualDevice                                          
0.0                     8842840  4438599.0  7771328     0.501943
1.0                       62690    12172.0    64634     0.194162

### Census_IsTouchEnabled

In [350]:
value_counts("Census_IsTouchEnabled")

train  train_sum     test  train_ratio
Census_IsTouchEnabled                                          
0                      7801452  3958835.0  6924286     0.507448
1                      1120031   500057.0   928967     0.446467

### Census_IsPenCapable

In [351]:
value_counts("Census_IsPenCapable")

train  train_sum     test  train_ratio
Census_IsPenCapable                                          
0                    8581834  4303801.0  7555266     0.501501
1                     339649   155091.0   297987     0.456621

### Census_IsAlwaysOnAlwaysConnectedCapable

In [352]:
value_counts("Census_IsAlwaysOnAlwaysConnectedCapable")

train  train_sum     test  \
Census_IsAlwaysOnAlwaysConnectedCapable                                
0.0                                      8341972  4232858.0  7340131   
1.0                                       508168   189287.0   422970   

                                         train_ratio  
Census_IsAlwaysOnAlwaysConnectedCapable               
0.0                                         0.507417  
1.0                                         0.372489

### Wdft_IsGamer

In [353]:
value_counts("Wdft_IsGamer")

train  train_sum     test  train_ratio
Wdft_IsGamer                                          
0.0           6174143  2976639.0  5302612     0.482114
1.0           2443889  1324330.0  2247784     0.541894

### Wdft_RegionIdentifier

In [354]:
value_counts("Wdft_RegionIdentifier")

train  train_sum     test  train_ratio
1.0   1232258   659826.0  1109642     0.535461
2.0     79385    36267.0    63098     0.456850
3.0   1295892   638119.0  1253395     0.492417
4.0    135567    64536.0    94853     0.476045
5.0    205372    99113.0   163548     0.482602
6.0    158163    75333.0   127072     0.476300
7.0    597297   299335.0   601077     0.501149
8.0    276029   135824.0   228934     0.492064
9.0     79882    36687.0    60802     0.459265
10.0  1800105   916641.0  1451816     0.509215
11.0  1347828   678230.0  1264670     0.503202
12.0   163711    77151.0   127190     0.471263
13.0   225130   107481.0   199668     0.477417
14.0     3822     2077.0    16864     0.543433
15.0  1017591   474349.0   787767     0.466149

In [355]:
to_categorical("Wdft_RegionIdentifier")

In [481]:
reduce_mem_usage(X_train)
reduce_mem_usage(X_test)

Mem. usage decreased to 1480.42 Mb (61.8% reduction)
Mem. usage decreased to 1303.16 Mb (61.8% reduction)


ProductName  EngineVersion  AppVersion  AvSigVersion  \
0                0.5       0.499756    0.499756          8266   
1                0.5       0.499756    0.497314          8266   
2                0.5       0.492432    0.497314          8266   
3                0.5       0.499756    0.499756          8266   
4                0.5       0.499756    0.497314          8266   
5                0.5       0.492432    0.497314          8266   
6                0.5       0.499756    0.499756          8266   
7                0.5       0.492432    0.485107          8266   
8                0.5       0.499756    0.499756          8266   
9                0.5       0.324219    0.317871          6281   
10               0.5       0.499756    0.499756          8266   
11               0.5       0.499756    0.472656          8266   
12               0.5       0.499756    0.499756          8266   
13               0.5       0.499756    0.499756          8266   
14               0.5       0.499756    0.499756          8266   
15               0.5       0.499756    0.499756          8266   
16               0.5       0.499756    0.499756          8266   
17               0.5       0.499756    0.457275          8266   
18               0.5       0.499756    0.499756          8266   
19               0.5       0.499756    0.499756          8266   
20               0.5       0.384521    0.529785          5109   
21               0.5       0.492432    0.457275          8266   
22               0.5       0.492432    0.497314          8266   
23               0.5       0.492432    0.499756          8266   
24               0.5       0.491699    0.444092          7815   
25               0.5       0.499756    0.499756          8266   
26               0.5       0.492432    0.497314          8266   
27               0.5       0.552734    0.530273          7722   
28               0.5       0.499756    0.530273          8266   
29               0.5       0.499756    0.499756          8266   
...              ...            ...         ...           ...   
7853223          0.5       0.492432    0.486328          8266   
7853224          0.5       0.499756    0.499756          8266   
7853225          0.5       0.499756    0.499756          8266   
7853226          0.5       0.499756    0.499756          8266   
7853227          0.5       0.499756    0.499756          8266   
7853228          0.5       0.492432    0.497314          8266   
7853229          0.5       0.499756    0.499756          8266   
7853230          0.5       0.491699    0.530273          8100   
7853231          0.5       0.499756    0.499756          8266   
7853232          0.5       0.499756    0.499756          8266   
7853233          0.5       0.492432    0.497314          8266   
7853234          0.5       0.492432    0.497314          8266   
7853235          0.5       0.499756    0.521973          8266   
7853236          0.5       0.309814    0.444580          6432   
7853237          0.5       0.499756    0.499756          8266   
7853238          0.5       0.492432    0.497314          8266   
7853239          0.5       0.492432    0.497314          8266   
7853240          0.5       0.492432    0.497314          8266   
7853241          0.5       0.499756    0.499756          8266   
7853242          0.5       0.492432    0.497314          8266   
7853243          0.5       0.302490    0.499756          6772   
7853244          0.5       0.499756    0.499756          8266   
7853245          0.5       0.492432    0.497314          8266   
7853246          0.5       0.492432    0.497314          8266   
7853247          0.5       0.492432    0.497314          8266   
7853248          0.5       0.499756    0.499756          8266   
7853249          0.5       0.499756    0.499756          8266   
7853250          0.5       0.499756    0.499756          8266   
7853251          0.5       0.499756    0.486328          8266   
7853252          0.5       0.499756    0.499

In [482]:
X_train.to_pickle("../result/X_train_20190209.pkl")
X_test.to_pickle("../result/X_test_20190209.pkl")
#tr_train = pd.read_pickle("../result/tr_train_20190207.pkl")
#X_test = pd.read_pickle("../result/X_test_20190207.pkl")

In [406]:
y_train = tr_train.HasDetections
X_train = tr_train.drop(['HasDetections'], axis=1)
del tr_train
ID_test = drop_id_n_get_test_id()

In [402]:

#ID_test = pd.read_pickle("../result/ID_test.pkl")

In [ ]:
#tr_train = pd.read_pickle("../result/tr_train_20190207.pkl")
#ID_test = pd.read_pickle("../result/ID_test.pkl")

In [ ]:
add_features()

In [474]:
category_columns = list(X_train.select_dtypes(include=['category']).columns)
label_columns = []
for column in category_columns:
    vc = value_counts_simple(column)
    if len(vc.index) > 1000:
        label_columns.append(column)

to_label_encode(label_columns)

C:\Users\southman\Anaconda3\lib\site-packages\pandas\core\frame.py:7397: RuntimeWarning: '<' not supported between instances of 'str' and 'float', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)
C:\Users\southman\Anaconda3\lib\site-packages\pandas\core\frame.py:7397: RuntimeWarning: '<' not supported between instances of 'float' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)


Encoded column: AVProductStatesIdentifier
Encoded column: CityIdentifier
Encoded column: Census_OEMNameIdentifier
Encoded column: Census_OEMModelIdentifier
Encoded column: Census_ProcessorModelIdentifier
Encoded column: Census_FirmwareVersionIdentifier
Encoded column: monitor_dims


(         ProductName  EngineVersion  AppVersion  AvSigVersion  \
 0           0.499958       0.552693    0.530385          7536   
 1           0.499958       0.437158    0.521794          6039   
 2           0.499958       0.552693    0.530385          7407   
 3           0.499958       0.552693    0.530385          7458   
 4           0.499958       0.552693    0.530385          7417   
 5           0.499958       0.552693    0.530385          7337   
 6           0.499958       0.552693    0.530385          7719   
 7           0.499958       0.552693    0.530385          7420   
 8           0.499958       0.491732    0.530385          8242   
 9           0.499958       0.552693    0.530385          7743   
 10          0.499958       0.552693    0.530385          7728   
 11          0.499958       0.552693    0.530385          7551   
 12          0.499958       0.552693    0.530385          7415   
 13          0.499958       0.491732    0.530385          8202   
 14       

In [475]:
to_target_encode_each()

Encoded column: CountryIdentifier
Encoded column: OrganizationIdentifier
Encoded column: GeoNameIdentifier
Encoded column: LocaleEnglishNameIdentifier
Encoded column: Platform
Encoded column: Processor
Encoded column: OsBuild
Encoded column: OsSuite
Encoded column: OsPlatformSubRelease
Encoded column: OsBuildLab
Encoded column: SkuEdition
Encoded column: PuaMode
Encoded column: IeVerIdentifier
Encoded column: SmartScreen
Encoded column: UacLuaenable
Encoded column: Census_MDC2FormFactor
Encoded column: Census_ProcessorManufacturerIdentifier
Encoded column: Census_ProcessorClass
Encoded column: Census_PrimaryDiskTypeName
Encoded column: Census_ChassisTypeName
Encoded column: Census_PowerPlatformRoleName
Encoded column: Census_OSVersion
Encoded column: Census_OSArchitecture
Encoded column: Census_OSBranch
Encoded column: Census_OSEdition
Encoded column: Census_OSSkuName
Encoded column: Census_OSInstallTypeName
Encoded column: Census_OSInstallLanguageIdentifier
Encoded column: Census_OSUI

(         ProductName  EngineVersion  AppVersion  AvSigVersion  \
 0           0.499958       0.552693    0.530385          7536   
 1           0.499958       0.437158    0.521794          6039   
 2           0.499958       0.552693    0.530385          7407   
 3           0.499958       0.552693    0.530385          7458   
 4           0.499958       0.552693    0.530385          7417   
 5           0.499958       0.552693    0.530385          7337   
 6           0.499958       0.552693    0.530385          7719   
 7           0.499958       0.552693    0.530385          7420   
 8           0.499958       0.491732    0.530385          8242   
 9           0.499958       0.552693    0.530385          7743   
 10          0.499958       0.552693    0.530385          7728   
 11          0.499958       0.552693    0.530385          7551   
 12          0.499958       0.552693    0.530385          7415   
 13          0.499958       0.491732    0.530385          8202   
 14       

In [433]:
corr_rank = X_train.corr().abs().unstack().sort_values(ascending = False)

In [450]:
#corr_rank[20:]

In [483]:
%%time
gc.collect()
SEED = 42
KFOLD = 5
models, validation_score = train_model(X_train, y_train, get_base_params(SEED),
                                      num_round=20000,
                                      early_stopping_rounds=100,
                                      save_feature_importances=True,
                                      random_state=SEED,
                                      number_of_folds = KFOLD,
                                      break_first=True)

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.702258	training's auc: 0.702258	valid_1's auc: 0.701812	valid_1's auc: 0.701812
[200]	training's auc: 0.709721	training's auc: 0.709721	valid_1's auc: 0.709087	valid_1's auc: 0.709087
[300]	training's auc: 0.714234	training's auc: 0.714234	valid_1's auc: 0.713404	valid_1's auc: 0.713404
[400]	training's auc: 0.717309	training's auc: 0.717309	valid_1's auc: 0.716299	valid_1's auc: 0.716299
[500]	training's auc: 0.719646	training's auc: 0.719646	valid_1's auc: 0.718471	valid_1's auc: 0.718471
[600]	training's auc: 0.721704	training's auc: 0.721704	valid_1's auc: 0.720376	valid_1's auc: 0.720376
[700]	training's auc: 0.723583	training's auc: 0.723583	valid_1's auc: 0.72211	valid_1's auc: 0.72211
[800]	training's auc: 0.725036	training's auc: 0.725036	valid_1's auc: 0.723422	valid_1's auc: 0.723422
[900]	training's auc: 0.726298	training's auc: 0.726298	valid_1's auc: 0.724549	valid_1's auc: 0.724549
[10

In [484]:
pred_test = get_predicts(models, X_test)

In [485]:
log.save2("southman features", X_train, get_base_params(SEED), ["lightGbm"], validation_score)

In [486]:
log.get(5)

id         train_name  train_models  train_data_cnt  \
0  74  southman features  ["lightGbm"]         8921483   
1  73  southman features  ["lightGbm"]         8921483   
2  72  southman features  ["lightGbm"]         8921483   
3  71  southman features  ["lightGbm"]         8921483   
4  70  southman features  ["lightGbm"]         8921483   

                                              params     score  \
0  {"boosting_type": "gbdt", "objective": "binary...  0.740854   
1  {"boosting_type": "gbdt", "objective": "binary...  0.727800   
2  {"boosting_type": "gbdt", "objective": "binary...  0.726996   
3  {"boosting_type": "gbdt", "objective": "binary...  0.724909   
4  {"boosting_type": "gbdt", "objective": "binary...  0.722731   

                                            features  features_cnt  \
0  ["ProductName", "EngineVersion", "AppVersion",...            81   
1  ["ProductName", "EngineVersion", "AppVersion",...            81   
2  ["ProductName", "EngineVersion", "AppVersion",...            84   
3  ["ProductName", "EngineVersion", "AppVersion",...            80   
4  ["ProductName", "EngineVersion", "AppVersion",...            80   

                                        train_sample     insert_datetime  \
0  {"schema": {"fields":[{"name":"index","type":"... 2019-02-09 14:20:38   
1  {"schema": {"fields":[{"name":"index","type":"... 2019-02-08 22:18:19   
2  {"schema": {"fields":[{"name":"index","type":"... 2019-02-07 23:09:25   
3  {"schema": {"fields":[{"name":"index","type":"... 2019-02-07 09:50:04   
4  {"schema": {"fields":[{"name":"index","type":"... 2019-02-07 03:08:23   

                                              source  
0  \n# coding: utf-8\n\n# In[1]:\n\n\nimport pand...  
1  \n# coding: utf-8\n\n# In[1]:\n\n\nimport pand...  
2  \n# coding: utf-8\n\n# In[1]:\n\n\nimport pand...  
3  \n# coding: utf-8\n\n# In[1]:\n\n\nimport pand...  
4  \n# coding: utf-8\n\n# In[1]:\n\n\nimport pand...

In [487]:
%%time
submission = pd.DataFrame({'MachineIdentifier':ID_test, 'HasDetections':pred_test})
filename = 'submission_{:.6f}_{}_folds_{}_data.csv'.format(validation_score,dt.now().strftime('%Y-%m-%d-%H-%M'), len(models))
submission.to_csv('../result/single_{}'.format(filename), index=False)

Wall time: 35.2 s


In [488]:
submission.head()

MachineIdentifier  HasDetections
0  0000010489e3af074adeac69c53e555e       0.446910
1  00000176ac758d54827acd545b6315a5       0.460497
2  0000019dcefc128c2d4387c1273dae1d       0.563374
3  0000055553dc51b1295785415f1a224d       0.355179
4  00000574cefffeca83ec8adf9285b2bf       0.507498